# KELOMPOK 4
## Muhammad Farhan Arya Wicaksono (5054231011)
## Fadhil zaky Budianto (5054231001)
## Efan Ramdhani (5054231017)

# Download Dataset ( Medical QA Indonesia ) 📕
---

Dataset yang digunakan dalam penelitian ini adalah Medical QA Indonesia, yang diperoleh dari repository publik di Mendeley Data . Dataset ini berisi 497.974 data mentah hasil pengumpulan dari platform alodokter.com, sebuah situs web konsultasi kesehatan daring gratis di Indonesia, yang mencakup data dari 8 Desember 2014 hingga 28 Februari 2021. Data ini terdiri dari teks-teks konsultasi kesehatan dalam bahasa Indonesia dan digunakan sebagai dasar pengetahuan (knowledge base) bagi sistem chatbot medis.

[ Medical QA Indonesia Mendeley data](https://data.mendeley.com/datasets/p8d5bynh3m/1)

In [ ]:
! wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/p8d5bynh3m-1.zip

--2025-06-29 07:23:00--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/p8d5bynh3m-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 3.5.70.120, 3.5.65.46, 52.218.61.152, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|3.5.70.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290368216 (277M) [application/zip]
Saving to: ‘p8d5bynh3m-1.zip’

p8d5bynh3m-1.zip    100%[===================>] 276.92M  14.9MB/s    in 21s     

2025-06-29 07:23:22 (13.2 MB/s) - ‘p8d5bynh3m-1.zip’ saved [290368216/290368216]



# Subsetting Data 📚

---

### Pemilihan dan Pemfilteran Data

Dataset yang digunakan merupakan kumpulan *Medical QA* berbahasa Indonesia. Dari dataset tersebut, dipilih kolom-kolom yang relevan untuk pelatihan model, yaitu:

* `title`
* `question`
* `answer`
* `topic_set`

Proses pemfilteran difokuskan pada **domain kesehatan pencernaan**. Kolom `topic_set` menjadi acuan utama dalam menyeleksi data yang relevan. Setiap entri dalam dataset akan dipilih jika nilai pada kolom `topic_set` mengandung salah satu dari kata kunci berikut:

* *gangguan-pencernaan*
* *asam-lambung*
* *tukak-lambung*
* *sakit-maag*
* *gastritis*
* *mual*
* *perut-kembung*
* *radang-usus*
* *diare*
* *konstipasi*
* *wasir*
* *sakit-perut*

Dengan menerapkan pemfilteran berdasarkan kata kunci tersebut, diperoleh subkorpus yang hanya terdiri dari data dengan topik kesehatan pencernaan. Hasil akhir dari proses ini menghasilkan sebanyak **21.376 baris data** yang siap digunakan untuk pelatihan model.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 1000)

In [ ]:
os.listdir( 'Doctors Answer Text Dataset in Indonesian Contains Information on Medical Interview Patterns')

['Indo-Online Health Consultation-Medical Interview-Clean.csv',
 'Indo-Online Health Consultation-Medical Interview-Label.csv',
 'Indo-Online Health Consultation-Multilabel-Raw.csv',
 'README.txt']

In [ ]:
raw_data = pd.read_csv( '/content/Doctors Answer Text Dataset in Indonesian Contains Information on Medical Interview Patterns/Indo-Online Health Consultation-Multilabel-Raw.csv')

In [ ]:
raw_data.shape

(497974, 10)

In [ ]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497974 entries, 0 to 497973
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   title           497974 non-null  object 
 1   question        497974 non-null  object 
 2   question_date   497974 non-null  object 
 3   answer          497974 non-null  object 
 4   answer_date     497974 non-null  object 
 5   topics          497974 non-null  object 
 6   topic_set       497974 non-null  object 
 7   risk            497974 non-null  object 
 8   year            497974 non-null  int64  
 9   time_to_answer  497974 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 38.0+ MB


In [ ]:
raw_data.sample(10)

,title,question,question_date,answer,answer_date,topics,topic_set,risk,year,time_to_answer
328129,mengatasi jerawat dan bekasnya,"Siang dok.. Beberapa minggu yg lalu saya memiliki jerawat jumbo (hampir seperti bisul) di pipi kiri saya dok. Skrg jerawat jumbo tadi sudah hilang, tetapi masih ada bekasnya di kulit dok, dan kalo...","16 January 2018, 14:52","Haloo Sdr. Niken, Jerawat (acne vulgaris) merupakan salah satu masalah yang sering dihadapi oleh wanita saat ini. Jerawat muncul karena peradangan kronis yang terjadi akibat adanya penumpukan min...","17 January 2018, 22:22",jerawat,jerawat,low,2018,1.0
461500,Penyebab benjolan lipoma di area panggul belakang,"Salam,Halo, Dok.Saya perempuan berusia 24 tahun ini. Saat SD kepala 5 atau umur 11 th saya memiliki benjolan di dalam daging, di panggul belakang atau sejajar dengan cekungan belakang dekat tulang...","20 July 2020, 16:12","Alo, terimakasih atas pertanyaannya. Lipoma adalah salah satu jenis tumor jinak berupa gumpalan berlemak yang terletak di antara kulit dan juga otot. Lipoma ini seringnya tidak berbahaya. Namun m...","20 July 2020, 18:10",benjolan lipoma,lipoma,low,2020,0.0
399944,Batasan konsumsi obat pereda sakit gigi,"Dok, saya mau tnya, apakah minum obat pereda nyeri sakit gigi musti rutin? Atau hnya ketika sedang merasakan sakitnya? Obat yg sama minum, Asam Mefenamat dan Dexamethason.. Terima kasih","6 September 2018, 07:40","Terimakasih telah bertanya di Alodokter. Kedua obat yang Anda sebutkan merupakan obat anti radang, yang sekaligus juga penggunaannya dapat mengurangi rasa nyeri. Umumnya, penggunaan obat ini hany...","6 September 2018, 10:17",sakit-gigi gigi,gigi,low,2018,0.0
142134,Cara mengatasi payudara bengkak dan keras pasca aborsi,"Hallo dok. Saya mau tanyak. Setelah aborsi payudara terasa bengkak dan keras. Bagaimana cara mengatasinya?Apakah itu ASI nya harus di keluar kan. Kalau harus di keluarkan, Bagaimana cara mengeluar...","4 October 2019, 10:52","Alo Putri, Sebelumnya perlu diketahui dulu bahwa aborsi tanpa indikasi medis dan hukum merupakan tindakan ilegal yang bisa dikenai sanksi hukum berupa penjara 10 tahun dan denda 1 milyar rupiah....","4 October 2019, 12:52",payudara keguguran,keguguran,low,2019,0.0
182467,"Makanan untuk penderita diabetes, jantung dan gangguan ginjal","Hallo dokter, saya ingin bertanya. Ayah saya penderita diabetes awalnya, belum lama ini luka basahnya baru sembuh selama perawatan sekitar 1 tahun. Kemudian ayah saya mendapat serangan jantung, se...","3 May 2018, 19:20","Halo NJ, Disarankan konsultasikan jenis diet sesui kondisi medis pasien ke dokter ahli gizi dengan membawa hasil laboratorium. Dokter ahli gizi akan menghitung kebutuhan kalori pasien dalam seha...","4 May 2018, 00:21",nutrisi,nutrisi,low,2018,0.0
207234,Haid berkepanjangan setelah menggunakan KM implan,Dokk saya haid hampir 2bln lebih Saya menggunakan kb implan dan saya setres juga dok Bahaya ngga dok Sya lagi menyusui,"5 February 2018, 21:26","Halo, Penggunaan alat kontrasepsi tentunya memiliki kekurangan dan kelebihannya masing-masing. Penggunaan KB implan dapat berdampak kepada siklus menstruasi dan tentunya hal ini berbeda-beda p...","6 February 2018, 20:25",menstruasi kontrasepsi,kontrasepsi,low,2018,0.0
219708,Penyebab belum hamil setelah 3 tahun menikah,"Hai dokter,,saya sudah 3 tahun menikah dan sudah melakukan banyak usaha untuk berhasil hamil. Salah satunya sudah sering ke dokter kandungan juga. Pertanyaan saya:1. Apakah kira2 penyebab hamil ta...","12 July 2020, 18:26","Alo Nur, Terima kasih atas pertanyaannya. Apakah dokter kandungan menyampaikan adanya gangguan pada organ reproduksi Anda? Apakah Anda masih teratur kontrol ke dokter kandungan? Pemeriksaan apa sa...","12 July 2020, 20:23",merencanakan-kehamilan endometriosis pcos,pcos,low,2020,0.0
320911,Penyebab posisi mata yang tidak sejajar,dok kenapa mata saya ga sejajar saat saya bengong/melamun dok? tapi saat tidak bengong/melamun mata saya sejajar dok,"6 August 2020, 20:17","Alo, terima

In [ ]:
raw_data.nunique()

,0
title,328303
question,357848
question_date,326650
answer,359672
answer_date,327795
topics,58153
topic_set,1541
risk,2
year,8
time_to_answer,128


In [ ]:
# Digestive Health Topics with counts
digestive_health_topics = [
    "gangguan-pencernaan",       # 7083
    "asam-lambung",              # 5891
    "tukak-lambung",             # 2952
    "sakit-maag",                # 2858
    "gastritis",                 # 2117
    "mual",                      # 1489
    "perut-kembung",             # 1279
    "radang-usus",               # 1291
    "diare",                     # 1302
    "konstipasi",                # 830
    "wasir",                     # 1851
    "sakit-perut",               # 763
]

# Total count: ~26,706

In [ ]:
digestive_data = raw_data[raw_data['topic_set'].isin(digestive_health_topics)]

In [ ]:
digestive_data = digestive_data.drop_duplicates()
digestive_data = digestive_data.reset_index(drop=True)
digestive_data.to_csv('digestive_data.csv', index=False)

## Dihasilkan data sebanyak 21.376

In [ ]:
digestive_data

,title,question,question_date,answer,answer_date,topics,topic_set,risk,year,time_to_answer
0,Penyebab dan cara mengatasi BAB berdarah,"Alo dok. Barusan saya BAB, tapi BAB nya keluar darah lumayan banyak. kejadian ini sudah terjadi sekitar 3-4 kalian. setiap saya BAB, pasti selalu ada darah, tapi sebelumnya hanya 1 tetes saja dara...","4 February 2021, 14:27","Alo Sania, terima kasih ya sudah bertanya di Alodokter. Keluhan yang anda alami dapat saja merupakan gejala dari : Disentri (bakteri ataupun amuba) Wasir Radang usus Luka pada anus Untuk membantu ...","4 February 2021, 15:35",wasir gangguan-pencernaan,gangguan-pencernaan,low,2021,0.0
1,Susah BAB setelah operasi wasir 3 bulan lalu,"Dok , pasca oprasi hemoroid 3 bulan lalu . Saya tidak pernah merasakan perut mulas ingin BAB .tapj lubang anus saya ada dorongan terus untuk BAB . Jika di ikuti fases yg kluar hanya sedikit tapi s...","9 February 2021, 11:09","Alo, Terima kasih atas pertanyaannya. Keluhan sulit BAB dengan riwayat operasi hemorrhoid dapat disebabkan oleh banyak hal. Hal ini tergantung sejak kapan keluhan ini dirasakan, gejala penyerta (n...","9 February 2021, 13:32",konstipasi wasir,wasir,low,2021,0.0
2,Bahaya menggunakan spekulum untuk mengatasi sulit bab pada penderita wasir,Saya sudah menderita wasir menahun dok.. saya sudah seminggu tidak bisa bab.. apakah membuka lubang anus dengan speculum untuk sekedar ingin bab lancar ... Apakah aman ?,"9 January 2021, 02:17","Alo, terimakasih atas pertanyaan anda di Alodokter. wasir atau dalam dunia kesehatan disebut sebagai hemoroid merupakan sutau pelebaran pembuluh darah di sekitar anus yang dapat mengakibatkan munc...","9 January 2021, 10:34",konstipasi wasir,wasir,low,2021,0.0
3,"Ambeien, kulit kepala iritasi, dan benjolan kecil di leher bagian belakang","Selamat Malam Dok, Saya mengalami Ambeyen, saya mengalami beberapa hal ini : BAB Berdarah berwarna merah terang, namun tidak nyeri. Gatal dan iritasi pada area sekitar saluran anus. Benjolan yang ...","10 January 2021, 20:03","Selamat malam, terima kasih telah bertanya di Alodokter. Ambeien, wasir atau hemorrhoid, adalah suatu bentuk kelainan pembuluh darah di area anus yang ditandai dengan munculnya benjolan dan bercam...","10 January 2021, 23:58",kulit benjolan wasir,wasir,low,2021,0.0
4,Penaganan susah BAB dan muncul benjolan pada anus,"Halo dok, pagi ini saya habis ke dokter saya masih kurang jelas apa kata sidokter habisnya keliahatannya nyebelin jadi saya hiraukan, saya punya keluhan susah bab, ketika bab sudah memucuk tapi ti...","21 January 2021, 10:48","Alo dicky terima kasih sudah bertanya kepada Alodokter saya mengerti akan kekhwatiran anda, dimana keluhan susah BAB dalam dunia medis disebut dengan konstipasi. dimana hal ini dapat disebabkan ka...","21 January 2021, 16:05",konstipasi wasir gangguan-pencernaan,gangguan-pencernaan,low,2021,0.0
...,...,...,...,...,...,...,...,...,...,...
21371,tanda penurunan asam lambung,"Dok, saya mau tanya lima hari yg lalu periksa kedokter katanya asam lambung saya naik. Gejala nya sih sesak napas, dikerongkongan kayak ada lendir diludahkan gak bisa keluar .tapi selama 6 hari in...","23 December 2014, 19:55","Hello,Asam lambung merupakan cairan yang diproduksi oleh sel-sel lambung yang berfungsi untuk mencerna protein, membunuh kuman yang masuk lambung dan sebagainya. Pada normalnya jumlah asam lambung...","31 December 2014, 14:58",asam-lambung,asam-lambung,low,2014,7.0
21372,mengatasi naiknya asam lambung,Dokter bagaimana cara mengatasi naikx asam lambumg sperti nyeri Ulu hati dan sesak dimana,"6 February 2015, 12:54","Hai,Asam lambung merupakan cairan yang akan dikeluarkan secara otomatis oleh sel sel pada lambung untuk mencerna makanan, membunuh kuman dll.Peningkatan cairan asam lambung biasa terjadi saat : ...","6 February 2015, 13:28",asam-lambung,asam-lambung,low,2015,0.0
21373,sering kembung dan buang gas,"dok mau tanya, perut saya sering kembung dan sering buang gas (kebanyakan di malam hari) apakah saya termasu

In [ ]:
!curl -F "reqtype=fileupload" -F "fileToUpload=@/content/digestive_data.csv" https://catbox.moe/user/api.php

https://files.catbox.moe/dyw9xs.csv

#Preprocessing Data 🧹
---

In [ ]:
! wget https://files.catbox.moe/dyw9xs.csv

In [ ]:
! unzip p8d5bynh3m-1.zip

Archive:  p8d5bynh3m-1.zip
  inflating: Doctors Answer Text Dataset in Indonesian Contains Information on Medical Interview Patterns/Indo-Online Health Consultation-Multilabel-Raw.csv  
  inflating: Doctors Answer Text Dataset in Indonesian Contains Information on Medical Interview Patterns/Indo-Online Health Consultation-Medical Interview-Label.csv  
  inflating: Doctors Answer Text Dataset in Indonesian Contains Information on Medical Interview Patterns/Indo-Online Health Consultation-Medical Interview-Clean.csv  
  inflating: Doctors Answer Text Dataset in Indonesian Contains Information on Medical Interview Patterns/README.txt  


In [ ]:
! wget https://files.catbox.moe/dyw9xs.csv

--2025-05-11 13:36:12--  https://files.catbox.moe/dyw9xs.csv
Resolving files.catbox.moe (files.catbox.moe)... 108.181.20.35, 2604:6600:5:1::3
Connecting to files.catbox.moe (files.catbox.moe)|108.181.20.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43562843 (42M) [application/octet-stream]
Saving to: ‘dyw9xs.csv’

dyw9xs.csv          100%[===================>]  41.54M  29.0MB/s    in 1.4s    

2025-05-11 13:36:14 (29.0 MB/s) - ‘dyw9xs.csv’ saved [43562843/43562843]



In [ ]:
! pip install indoNLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import indoNLP

from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
df = pd.read_csv('dyw9xs.csv')
df.head()

,title,question,question_date,answer,answer_date,topics,topic_set,risk,year,time_to_answer
0,Penyebab dan cara mengatasi BAB berdarah,"Alo dok. Barusan saya BAB, tapi BAB nya keluar darah lumayan banyak. kejadian ini sudah terjadi sekitar 3-4 kalian. setiap saya BAB, pasti selalu ada darah, tapi sebelumnya hanya 1 tetes saja darahnya. sekarang darahnya banyak, warnanya merah gelap (seperti darah haid). sejauh ini saya tidak merasa demam, diare, ataupun gejala lainnya. BAB saya juga terbilang cukup normal. kira kira itu kenapa ya dok? terimakasih","4 February 2021, 14:27","Alo Sania, terima kasih ya sudah bertanya di Alodokter. Keluhan yang anda alami dapat saja merupakan gejala dari : Disentri (bakteri ataupun amuba) Wasir Radang usus Luka pada anus Untuk membantu menentukan apa kemungkinan penyebab keluhan anda, ada baiknya anda juga memerhatikan apakah terdapat benjolan yang keluar dari anus, apakah darah bercampur dengan feses, dan terasa nyeri pada saat atau setelah BAB. Apabila keluhan saat ini masih berulang dan memang banyak, ada baiknya anda memeriksakan keluhan secara langsung ke dokter untuk dilakukan evaluasi dan penanganan pertama. Anda tidak disarankan mengonsumsi obat-obatan apapun tanpa adanya anjuran yang jelas dari dokter anda, terutama obat-obatan untuk menghentikan perdarahan. Anda juga disarankan untuk menghindari mengejan terlalu sering, meningkatkan konsumsi sayur dan buah, memastikan makanan yang anda konsumsi bersih dan matang sempurna serta menjaga asupan cairan minimal 2 liter perharinya. Apabila muncul keluhan seperti pusi...","4 February 2021, 15:35",wasir gangguan-pencernaan,gangguan-pencernaan,low,2021,0.0
1,Susah BAB setelah operasi wasir 3 bulan lalu,"Dok , pasca oprasi hemoroid 3 bulan lalu . Saya tidak pernah merasakan perut mulas ingin BAB .tapj lubang anus saya ada dorongan terus untuk BAB . Jika di ikuti fases yg kluar hanya sedikit tapi sering. Apa yg harus syaa lakukan ?","9 February 2021, 11:09","Alo, Terima kasih atas pertanyaannya. Keluhan sulit BAB dengan riwayat operasi hemorrhoid dapat disebabkan oleh banyak hal. Hal ini tergantung sejak kapan keluhan ini dirasakan, gejala penyerta (nyeri perut, nyeri pada anus, BAB keluar darah, diare, dll), riwayat pengobatan, dan lainnya. Terdapat beberapa kemungkinan penyebab dari keluhan yang Anda rasakan, seperti konstipasi/obstipasi, komplikasi pasca operasi, inkontinensia alvi, maupun irritable bowel syndrome . Apabila keluhan sulit BAB ini sudah terjadi setidaknya dalam 2 minggu terakhir ini, sebaiknya periksakan diri ke dokter. Dokter akan mengevaluasi keluhan Anda lebih lengkap lagi, melakukan pemeriksaan fisik, dan pemeriksaan penunjang yang relevan sehingga dokter dapat menentukan penyebab pasti dari keluhan Anda dan memberikan terapi yang sesuai. Untuk sementara waktu, berikut beberapa anjuran yang dapat Anda lakukan, seperti: Mencukupi kebutuhan nutrisi yang sehat dan bergizi. Mengonsumsi makanan yang kaya akan serat, ...","9 February 2021, 13:32",konstipasi wasir,wasir,low,2021,0.0
2,Bahaya menggunakan spekulum untuk mengatasi sulit bab pada penderita wasir,Saya sudah menderita wasir menahun dok.. saya sudah seminggu tidak bisa bab.. apakah membuka lubang anus dengan speculum untuk sekedar ingin bab lancar ... Apakah aman ?,"9 January 2021, 02:17","Alo, terimakasih atas pertanyaan anda di Alodokter. wasir atau dalam dunia kesehatan disebut sebagai hemoroid merupakan sutau pelebaran pembuluh darah di sekitar anus yang dapat mengakibatkan munculnya benjola, perdarahan dan juga nyeri saat BAB. umumnya hemoroid di bedakan menjadi beberapa tingkatan, diantaranya : grade 1 : BAB berdarah benjolan belum muncul grade 2 : BAB berdarah, benjolan keluar masuk sendiri grade 3 : BAB berdarah, benjolan keluar dan tidak bisa masuk sendiri tanpa bantuan jari grade 4 : BAB berdarah, benjolan tidak bisa keluar masuk meskipun dengan bantuan jari jika berdasarkan dari keluhan yang anda sampaikan, wasir anda kemungkinan Sudah mencapai grade 3-4. yang man

### drop kolom yang tidak dipakai

In [ ]:
df.drop(columns=['question_date', 'answer_date', 'time_to_answer', 'risk', 'year' ], inplace=True)
df.head()

,title,question,answer,topics,topic_set
0,Penyebab dan cara mengatasi BAB berdarah,"Alo dok. Barusan saya BAB, tapi BAB nya keluar darah lumayan banyak. kejadian ini sudah terjadi sekitar 3-4 kalian. setiap saya BAB, pasti selalu ada darah, tapi sebelumnya hanya 1 tetes saja darahnya. sekarang darahnya banyak, warnanya merah gelap (seperti darah haid). sejauh ini saya tidak merasa demam, diare, ataupun gejala lainnya. BAB saya juga terbilang cukup normal. kira kira itu kenapa ya dok? terimakasih","Alo Sania, terima kasih ya sudah bertanya di Alodokter. Keluhan yang anda alami dapat saja merupakan gejala dari : Disentri (bakteri ataupun amuba) Wasir Radang usus Luka pada anus Untuk membantu menentukan apa kemungkinan penyebab keluhan anda, ada baiknya anda juga memerhatikan apakah terdapat benjolan yang keluar dari anus, apakah darah bercampur dengan feses, dan terasa nyeri pada saat atau setelah BAB. Apabila keluhan saat ini masih berulang dan memang banyak, ada baiknya anda memeriksakan keluhan secara langsung ke dokter untuk dilakukan evaluasi dan penanganan pertama. Anda tidak disarankan mengonsumsi obat-obatan apapun tanpa adanya anjuran yang jelas dari dokter anda, terutama obat-obatan untuk menghentikan perdarahan. Anda juga disarankan untuk menghindari mengejan terlalu sering, meningkatkan konsumsi sayur dan buah, memastikan makanan yang anda konsumsi bersih dan matang sempurna serta menjaga asupan cairan minimal 2 liter perharinya. Apabila muncul keluhan seperti pusi...",wasir gangguan-pencernaan,gangguan-pencernaan
1,Susah BAB setelah operasi wasir 3 bulan lalu,"Dok , pasca oprasi hemoroid 3 bulan lalu . Saya tidak pernah merasakan perut mulas ingin BAB .tapj lubang anus saya ada dorongan terus untuk BAB . Jika di ikuti fases yg kluar hanya sedikit tapi sering. Apa yg harus syaa lakukan ?","Alo, Terima kasih atas pertanyaannya. Keluhan sulit BAB dengan riwayat operasi hemorrhoid dapat disebabkan oleh banyak hal. Hal ini tergantung sejak kapan keluhan ini dirasakan, gejala penyerta (nyeri perut, nyeri pada anus, BAB keluar darah, diare, dll), riwayat pengobatan, dan lainnya. Terdapat beberapa kemungkinan penyebab dari keluhan yang Anda rasakan, seperti konstipasi/obstipasi, komplikasi pasca operasi, inkontinensia alvi, maupun irritable bowel syndrome . Apabila keluhan sulit BAB ini sudah terjadi setidaknya dalam 2 minggu terakhir ini, sebaiknya periksakan diri ke dokter. Dokter akan mengevaluasi keluhan Anda lebih lengkap lagi, melakukan pemeriksaan fisik, dan pemeriksaan penunjang yang relevan sehingga dokter dapat menentukan penyebab pasti dari keluhan Anda dan memberikan terapi yang sesuai. Untuk sementara waktu, berikut beberapa anjuran yang dapat Anda lakukan, seperti: Mencukupi kebutuhan nutrisi yang sehat dan bergizi. Mengonsumsi makanan yang kaya akan serat, ...",konstipasi wasir,wasir
2,Bahaya menggunakan spekulum untuk mengatasi sulit bab pada penderita wasir,Saya sudah menderita wasir menahun dok.. saya sudah seminggu tidak bisa bab.. apakah membuka lubang anus dengan speculum untuk sekedar ingin bab lancar ... Apakah aman ?,"Alo, terimakasih atas pertanyaan anda di Alodokter. wasir atau dalam dunia kesehatan disebut sebagai hemoroid merupakan sutau pelebaran pembuluh darah di sekitar anus yang dapat mengakibatkan munculnya benjola, perdarahan dan juga nyeri saat BAB. umumnya hemoroid di bedakan menjadi beberapa tingkatan, diantaranya : grade 1 : BAB berdarah benjolan belum muncul grade 2 : BAB berdarah, benjolan keluar masuk sendiri grade 3 : BAB berdarah, benjolan keluar dan tidak bisa masuk sendiri tanpa bantuan jari grade 4 : BAB berdarah, benjolan tidak bisa keluar masuk meskipun dengan bantuan jari jika berdasarkan dari keluhan yang anda sampaikan, wasir anda kemungkinan Sudah mencapai grade 3-4. yang mana penangana hemrorid grade 3-4 yaitu haruslah dengan penangan medis dokter yaitu dengan operasi. karena jika tidak di operasi maka akan mengakibatkan, anemia, trmbosis, keruskan pada anus, infeksi. ole

### Pola nama

Pada answer maupun question. terdapat nama yang mana harus di hapus. terutama pada awal kalimat pada suatu paragraf

Jika kita mengambil sample 200 data kemudain kita cek untuk kalimat pertamanya. Bisa dilihat kalo ada beberapa pola untuk mempermudah prepocessing

1. **Sapaan Umum (tanpa nama)**:
   - Contoh:
     - "Halo,"
     - "Hai,"
     - "Salam,"
     - "Alo,"
     - "Selamat [waktu],"
   - Variasi: Beberapa sapaan diikuti langsung dengan isi atau penjelasan.

2. **Sapaan dengan Nama**:
   - Contoh:
     - "Halo [Nama],"
     - "Hai [Nama],"
     - "Alo [Nama],"
     - "Dear [Nama],"
     - "[Nama] yang baik,"
   - Variasi: Nama bisa berupa nama lengkap, nama panggilan, atau nama identitas.

3. **Sapaan dengan Terima Kasih**:
   - Contoh:
     - "Halo, terima kasih atas pertanyaannya."
     - "Halo [Nama], terimakasih atas pertanyaan Anda."
     - "Alo [Nama], terima kasih sudah bertanya."
     - "Hai, terimakasih telah bertanya di Alodokter."
   - Variasi: Ungkapan terima kasih sering muncul setelah sapaan.

4. **Sapaan dengan Pendahuluan Informasi**:
   - Contoh:
     - "Halo [Nama], keluhan yang Anda sampaikan..."
     - "Hai [Nama], dari keluhan yang Anda jabarkan..."
     - "Halo, nyeri perut yang Anda alami..."
     - "Dear [Nama], gejala yang Anda alami..."
   - Variasi: Langsung menuju pokok pembahasan setelah sapaan.

5. **Sapaan dengan Format Salam Islami**:
   - Contoh:
     - "Wa'alaikumsalam [Nama],"
     - "Wa'alaikumsalam wr wb, Halo [Nama],"
   - Variasi: Digunakan salam Islami untuk pembuka.

6. **Sapaan Singkat atau Cepat**:
   - Contoh:
     - "Hai."
     - "Halo."
     - "Alo!"
     - "Salam."

- Halo  [person name],
- Dear [Person name],
- Hai [Person name],
- Hallo  [person name],
- Dear [person name],
- Alo [person name],
- hello [person name],
- salam [person name],
- hay ['person name']

---
- Halo  [person name].
- Dear [Person name].
- Hai [Person name].
- Hallo  [person name].
- Dear [person name]
---
- Hai,
- Halo,
- hello,
---

- Selamat pagi/siang/malam [person name].
- Selamat pagi/siang/malam [person name],
- Selamat pagi/siang/malam [person name]

---

- Hai.
- Halo.
- Alo.
---
- Halo [person name]..........
- Hai [person name]...........
- Halo [person name]..... Terimakasih atas pertanyaan Anda.
---
- Alo, terimakasih atas pertanyaannya.
- Halo terima kasih atas pertanyaannya untuk Alodokter.

---
-  Terima kasih [person name] atas pertanyaannya

- Terima kasih [person name] atas pertanyaannya.

- Terima kasih telah bertanya di Alodokter.com

- terimakasih atas pertanyaannya

In [ ]:
first_sentences = df['answer'].sample(200, random_state=42).apply(lambda x: sent_tokenize(x)[0] if x else "")
for idx, i in enumerate(first_sentences.values):
  print(f'{idx+1}. {i}')

1. Halo,Terimakasih atas pertanyaan yang Anda ajukan.Sebaiknya disebutkan kembali usia bayi Anda saat ini.
2. Halo Desriana......Terimakasih atas pertanyaan Anda.Nyeri pada daerah lambung biasanya disebut juga sebagai nyeri ulu hati atau dalam bahasa medis disebut sebagai nyeri epigastrium.
3. hai Anisnuri,terimakasih telah bertanya di alodokter.
4. Halo AiraniBatuk sebenernya adalah suatu reflek normal untuk membersihkan saluran nafas dari hal-hal yang tidak seharusnya ada di sana, seperti lendir, benda asing, atau kuman yang masuk.
5. Halo Febby, terima kasih atas pertanyaannya.
6. Alo, terimakasih atas pertanyaannya.
7. Halo Karuniadi, Terima kasih atas pertanyaannya.
8. HaloDari keluhan yang dipaparkan kemungkinan penyebabnya adalah karena ada luka di lambung Anda.
9. Virginia yang baik, keluhan yang Anda kemukakan belum mengarah pada satu penyakit yang spesifik.
10. ALo, Terimakasih atas pertanyaan anda di Alodokter Seseorang mengalami atau merasakan sesuatu yang mengganjal di ten

### Pola Lengkap:
1. **Sapaan**:
   - "Halo [Nama],"
   - "Hai [Nama],"
   - "Alo [Nama],"
   - "[Nama] yang baik,"
   - "Wa'alaikumsalam [Nama],"
2. **Ucapan Terima Kasih (opsional)**:
   - "Terima kasih atas pertanyaannya."
   - "Terimakasih telah bertanya di Alodokter."
3. **Pendahuluan Informasi (opsional)**:
   - "Keluhan yang Anda sampaikan..."
   - "Gejala yang Anda alami..."
   - "Dari keluhan yang dijabarkan..."

### Kombinasi:
- "Halo [Nama], terima kasih atas pertanyaannya."
- "[Nama] yang baik, keluhan yang Anda sampaikan..."
- "Alo [Nama], terimakasih telah bertanya di Alodokter."
- "Wa'alaikumsalam [Nama], gejala yang Anda alami..."

Tantangan nya yaitu ketika data seperti Halo/helo/hai [person name] tapi tidak ada pemisah seperti titik dan koma


Untuk pembuktian perlu dilakukan pengecekan dan perhitungan

In [ ]:
import re

def hitung_sapaan(teks):
    variants = r'(Hai|Hallo|Halo|Helo|Hello|Alo|Dear|Salam|Hay)'
    pola = rf'\b{variants}(?:\s+\w+){{0,5}}[,.!?]'
    hasil = re.findall(pola, teks, flags=re.IGNORECASE)
    return 1 if hasil else 0


In [ ]:
df['pola 1'] = df['answer'].apply(hitung_sapaan)

In [ ]:
df['pola 1'].value_counts()

,count
pola 1,
1,17706
0,3670


In [ ]:
df['pola 1'].value_counts(normalize = True)

,proportion
pola 1,
1,0.828312
0,0.171688


### Pola kalimat awal di data

Dari pola yang diberikan, berikut adalah pola umum untuk semua kalimat awal di data :

1. **[Person] yang baik**:
   - Contoh: "Nita yang baik", "Putrii yang baik", "Ainul yang baik", "Cindy Agustin yang baik".

2. **Selamat [waktu] [Person]**:
   - Contoh: "Selamat pagi pak Budi", "Selamat malam Najah Najmia Halim", "Selamat siang".

3. **Waalaikumsalam [Person]**:
   - Contoh: "Waalaikumsalam Afidah", "Waalaikumsalam wr wb".

4. **Terima kasih [Person] atas pertanyaannya**:
   - Contoh: "Terima kasih Furqon Indrayono atas pertanyaannya", "Terima kasih Jeehan Fatiha Yeoja Kpopers atas pertanyaannya".

5. **Hay [Person], terimakasih atas pertanyaannya**:
   - Contoh: "hay okta, terimakasih atas pertanyaannya".

6. **Sapaan umum tanpa menyebut nama**:
   - Contoh: "Selamat pagi", "Selamat malam", "Selamat sore".

Pola yang dapat dirumuskan:
1. **Sapaan + Nama atau Identitas**:
   - "[Person] yang baik,"
   - "Selamat [waktu] [Person]"
   - "Waalaikumsalam [Person]"
   - "Terima kasih [Person] atas pertanyaannya"
   - "Hay [Person], terimakasih atas pertanyaannya"

2. **Sapaan umum tanpa nama**:
   - "Selamat [waktu]"

### Pola check pola [person name] yang baik

In [ ]:
def namaYangBaik(teks):
    # Pola untuk menangkap nama diikuti oleh "yang baik"
    pola = r'\b([A-Z][a-z]+)\s+yang baik'

    # Mencocokkan pola dalam teks
    hasil = re.search(pola, teks)

    # Jika pola ditemukan, kembalikan nama (group pertama), jika tidak kembalikan None
    return 1 if hasil else 0

In [ ]:
df['pola 2'] = df['answer'].apply(namaYangBaik)

In [ ]:
df['pola 2'].value_counts()

,count
pola 2,
0,20937
1,439


 ada 400 an

In [ ]:
df[df['pola 2'] == 1].sample(30, random_state=2)

,title,question,answer,topics,topic_set,pola 1,pola 2
16618,Penyebab perut bagian kiri bawah terasa nyeri,"Pagi Dok.... saya mau bertanya,,kenapa yah perut bag kiri bawah saya agak sakit ??","Stevani yang baik, sakit perut merupakan salah satu kondisi yang sering ditemukan pada layanan primer. Sakit perut menjadi sedemikian kompleks oleh karena beragamnya organ yang menempati rongga perut yang sangat besar. Pada wanita, hal ini diperumit dengan adanya organ reproduksi yang cukup besar yakni rahim dan komponennya yang bila bermasalah, tentu menimbulkan sakit. Adapun beberapa kondisi nyeri perut pada bagian kiri bawah dapat disebabkan oleh : batu atau infeksi pada sistem ginjal batu atau infeksi pada saluran kemih usus buntu yang tidak khas dapat muncul dari kiri pankreatitis atau radang pada pankreas kista indung telur yang terplintir endometriosis atau adanya jaringan rahim diluar dinding rahim itu sendiri divertikulitis masalah pada leher rahim Adapun nyeri untuk sementara dapat diatasi dengan penahan nyeri yang dijual bebas seperti paracetamol atau ibuprofen tentunya dengan memperhatikan petunjuk pemakaian yang ada. Perhatikan kondisi lain yang menyertai perut sep...",gangguan-pencernaan,gangguan-pencernaan,0,1
15496,Sakir maag disertai muntah dan demam menggigil,asslamualaikum dok sy muklis 26 th mau tanya dok saya sudah seminggu sakit awal nya saya magh dok muntah-muntah sampai muntah kuning dok sampai demam menggil trus sy minum obat promagh dan paracetamol dok sdh baikan tp setelahx itu bsoknya saya malah sering kepla sakit dok badan nyeri-nyeri leher tegang dan di ikuti demam dok di jam\ tertentu..,"Muhammad yang baik, dalam mengamati satu penyakit hendaknya Anda tidak berpatokan pada satu gejala saja seperti gejala maag, oleh karena kondisi serupa dapat ditemui pada kondisi lain. Seperti misalnya petunjuk berupa rasa demam, hal ini lebih banyak mencerminkan kondisi infeksi oleh karena Indonesia yang berada pada daerah tropis, apalagi disertai gejala nyeri-nyeri sekujur tubuh yang merupakan gejala umum infeksi. Demam yang disertai dengan gejala maag biasanya mengacu pada infeksi saluran pencernaan seperti misalnya tifus, atau gejala diare. Penanganannya pun harus secara komprehensif agar pemulihan lebih efektif terjadi. Demam yang ada bisa dikurangi dengan penurun panas yang dijual bebas seperti paracetamol tentunya dengan memperhatikan petunjuk pemakaian yang ada. Maag yang ada dapat dikurangi dengan gaya hidup ramah lambung seperti mengurangi asupan makanan pedas/bersantan atau minuman berkafein seperti kopi, kemudian memperbaiki pola makan tidak teratur, dan manajemen str...",gangguan-pencernaan,gangguan-pencernaan,0,1
19536,Mengatasi penyakit asam lambung,"Hallo Dok..aku setiap minggu sering bgt diare, mual2 disertai muntah2, perut serasa seperti diperas2, mulut rasa nya mati rasa, tiap makan sesuatu ga berasa apa2 cuma asam dan mual, dan bagian belikat/punggung kiri atas sakit sekali seperti ada gelembungan angin menekan2 sampai saya angkat tangan kiri saja nyeri2 gitu..jujur saja, saya memang sering telat makan, krn kondisi kerja saya yg ga tentu ini, juga saya ini ngerokok tp bukan perokok besar, cuma kalo tiap plg kerja saja 2-3 batang atau kalo lg stress..dan saya pecinta kopi berat..apa lagi ice kopi..so yg ingin saya tanya kan apa sakit saya ini dok..kata orang2 itu asam lambung..kalo memang asam lambung apa ada obat yg manjur untuk mengobati sakit saya ini..dan apa yg harus saya lakukan agar tiap minggu ga sakit2 seperti ini terus dok..so bantu saya ya dok..terima kasih sebelumnya ya dok..","Lin yang baik, masalah pada pencernaan memang dapat muncul dalam berbagai bentuk, mulai dari rasa mual, penuh di lambung, muntah-muntah, nyeri perut hingga diare. Namun dari kondisi-kondisi tersebut setidaknya dapat diperkirakan penyebab kondisi yang ada, untuk kemudian dilakukan pemeriksaan lebih lanjut untuk menentukan diagnosis serta penanganannya. Adapun diare dan sakit lambung biasanya disebabkan oleh 2 hal yang ber

### Prepocessing Membersihkan sapaan
Dari hasil observasi sebelumnya didaptkan pola

In [ ]:
import re
import html

def bersihkan_sapaan(teks):

    # Decode \u0026lt; -> <, \u0026gt; -> >, dll
    teks = teks.replace('\\u0026lt;', '<').replace('\\u0026gt;', '>').replace('\\u0026', '&')
    teks = html.unescape(teks)
    # Hapus tag HTML beserta atribut (contoh: <span style="...">)
    teks = re.sub(r'<[^>]+>', '', teks)

    # 1. [Person] yang baik, contoh: "Nita yang baik," "Cindy Agustin yang baik,"
    teks = re.sub(
        r'^\s*([\w.]+(?:\s+[\w.]+){0,3})\s+yang\s+baik[.,:]*\s*', '', teks, flags=re.IGNORECASE)

    # 2. Selamat [waktu] [Person] (dengan/atau tanpa nama)
    teks = re.sub(
        r'^\s*Selamat\s+(pagi|siang|malam|sore)\s*([\w.]+(?:\s+[\w.]+){0,4})?[.,:]*\s*',
        '', teks, flags=re.IGNORECASE)

    # 3. Waalaikumsalam [Person]
    teks = re.sub(
        r'^\s*Waalaikumsalam(?:\s+wr\s+wb)?\s*([\w.]+(?:\s+[\w.]+){0,4})?[.,:]*\s*',
        '', teks, flags=re.IGNORECASE)

    # 4. Terima kasih [Person] atas pertanyaannya
    teks = re.sub(
        r'^\s*Terima\s+kasih\s+([\w.]+(?:\s+[\w.]+){0,4})\s+atas\s+pertanyaannya[.,:]*\s*',
        '', teks, flags=re.IGNORECASE)

    # 5. Hay [Person], terimakasih atas pertanyaannya
    teks = re.sub(
        r'^\s*Hay\s+([\w.]+(?:\s+[\w.]+){0,4}),?\s*terima[ -]?kasih\s+atas\s+pertanyaannya[.,:]*\s*',
        '', teks, flags=re.IGNORECASE)

    # 6. Sapaan umum tanpa nama: Selamat [waktu]
    teks = re.sub(
        r'^\s*Selamat\s+(pagi|siang|malam|sore)[.,:]*\s*', '', teks, flags=re.IGNORECASE)

    # 7. Variasi sapaan umum (Hai, Hallo, Halo, Helo, Hello, Alo, salam, Dear)
    variants = r'(Hai|Hallo|Halo|Helo|Hello|Alo|salam|Dear)'
    teks = re.sub(
        rf'^\s*{variants}(?:\s+\w+){{1,3}}[.,:]*', '', teks, flags=re.IGNORECASE)
    teks = re.sub(
        rf'^\s*{variants}[.,:]*', '', teks, flags=re.IGNORECASE)

    # # 8. Hapus "ke/di/pada alodokter" atau "alodokter" di awal/akhir/kapan pun
    # teks = re.sub(r'\balodokter\b', '', teks, flags=re.IGNORECASE)
    # teks = re.sub(r'\balodokter.com\b', '', teks, flags=re.IGNORECASE)
    # teks = re.sub(r'\balodokter\b', '', teks, flags=re.IGNORECASE)

    # Hapus frasa seperti "ke alodokter", "di alodokter", "pada alodokter", termasuk .com
    teks = re.sub(r'\b(ke|di|pada)\s+alodokter(?:\.com)?\b', '', teks, flags=re.IGNORECASE)

# Hapus kata "alodokter" atau "alodokter.com" jika berdiri sendiri
    teks = re.sub(r'\balodokter(?:\.com)?\b', '', teks, flags=re.IGNORECASE)


    # 9. Hapus "Dr. Nama Nama"
    teks = re.sub(r'\.?dr\.?\s*[A-Z][a-z]+(?:\s+[A-Z][a-z]+)?', '', teks, flags=re.IGNORECASE)
    teks = re.sub(r'\bDr\.\s+\w+\s+\w+\b', 'Dr.', teks, flags=re.IGNORECASE)
    teks = re.sub(r'\bDr\.\s+\w+\s+\w+\b', '', teks, flags=re.IGNORECASE)

    # 10. Bersihkan spasi ganda dan strip spasi awal/akhir
    teks = re.sub(r'\s+', ' ', teks).strip()

    return teks


### Contoh kasus lain

In [ ]:
"Hallo Rehan Putra Tunggal Terimakasih atas pertanyaan anda Jadi anda harus paham, bahwa asam lambung merupakan material yang diciptakan secara alamiah oleh lambung untuk membantu proses pencernaan makanan. Jadi tidak mungkin untuk menghilangkan asam lambung ini. Mungkin yang anda maksud adalah keluhan yang muncul akibat kondisi asam lambung berlebih/peningkatan asam lambung atau bisa juga karena penyakit asam lambung (GERD). Jika yang anda maksud adalah kedua kondisi ini, maka memang bisa dilakukan penanganan sehingga kondisi ini tidak menimbulkan keluhan. Penyakit asam lambung atau Gastroesophageal Reflux Disease (GERD) yang anda alami merupakan kondisi dimana terjadi nyeri pada ulu hati atau sensasi terbakar di dada akibat naiknya asam lambung menuju esophagus (kerongkongan). Penyakit asam lambung atau GERD pada umumnya disebabkan oleh tidak berfungsinya lower esophageal sphinchter (LES) yang merupakan lingkaran otot pada bagian bawah dari esophagus (kerongkongan). LES berf..."

'Hallo Rehan Putra Tunggal Terimakasih atas pertanyaan anda Jadi anda harus paham, bahwa asam lambung merupakan material yang diciptakan secara alamiah oleh lambung untuk membantu proses pencernaan makanan. Jadi tidak mungkin untuk menghilangkan asam lambung ini. Mungkin yang anda maksud adalah keluhan yang muncul akibat kondisi asam lambung berlebih/peningkatan asam lambung atau bisa juga karena penyakit asam lambung (GERD). Jika yang anda maksud adalah kedua kondisi ini, maka memang bisa dilakukan penanganan sehingga kondisi ini tidak menimbulkan keluhan. Penyakit asam lambung atau Gastroesophageal Reflux Disease (GERD) yang anda alami merupakan kondisi dimana terjadi nyeri pada ulu hati atau sensasi terbakar di dada akibat naiknya asam lambung menuju esophagus (kerongkongan). Penyakit asam lambung atau GERD pada umumnya disebabkan oleh tidak berfungsinya lower esophageal sphinchter (LES) yang merupakan lingkaran otot pada bagian bawah dari esophagus (kerongkongan). LES berf...'

In [ ]:
bersihkan_sapaan("Terima kasih Memey SiComell Pesekz atas pertanyaannya di forum Alodokter, keadaan yang mungkin Saudari gambarkan saat ini kemungkinan disebabkan oleh keadaan perut kembung. Perut kembung sendiri merupakan gangguan pada pencernaan dimana penderita merasa perut tidak nyaman karena terasa penuh atau begah. Pada keadaan ini dapat timbul gejala berupa: - sensasi penuh, sesak, atau pembengkakan pada bagian perut - rasa nyeri dan penuh gas pada perut - rasa sakit yang muncul bisa menyebabkan kram perut - sering bersendawa - perut terasa bergejolak - buang angin berlebihan Perut kembung dapat disebabkan oleh banyak hal, diantaranya seperti: - mengonsumsi makanan yang dapat memproduksi gas - menelan udara, baik saat makan maupun ketika minum air - sembelit - merokok - makan terlalu cepat - mengalami gangguan pada sistem pencernaan Makan-makanan tertentu umumnya dianggap bertanggung jawab atas timbulnya gas pada saluran cerna, gas ini umumnya dihasilkan dari hasil pemecahan makanan oleh bakt.")

'di forum , keadaan yang mungkin Saudari gambarkan saat ini kemungkinan disebabkan oleh keadaan perut kembung. Perut kembung sendiri merupakan gangguan pada pencernaan dimana penderita merasa perut tidak nyaman karena terasa penuh atau begah. Pada keadaan ini dapat timbul gejala berupa: - sensasi penuh, sesak, atau pembengkakan pada bagian perut - rasa nyeri dan penuh gas pada perut - rasa sakit yang muncul bisa menyebabkan kram perut - sering bersendawa - perut terasa bergejolak - buang angin berlebihan Perut kembung dapat disebabkan oleh banyak hal, diantaranya seperti: - mengonsumsi makanan yang dapat memproduksi gas - menelan udara, baik saat makan maupun ketika minum air - sembelit - merokok - makan terlalu cepat - mengalami gangguan pada sistem pencernaan Makan-makanan tertentu umumnya dianggap bertanggung jawab atas timbulnya gas pada saluran cerna, gas ini umumnya dihasilkan dari hasil pemecahan makanan oleh bakt.'

In [ ]:
bersihkan_sapaan('Pagi Jhunny,Terima kasih atas pertanyaannya.Keluhan berupa sakit pada perut yang dirasakan setiap makan tentunya dapat terasa mengganggu. Hal ini dapat disebabkan oleh hal berikut: Gastritis , yaitu \u0026lt;span style=\color: #000000; font-family: museosans300, sans-serif;\\u0026gt;kondisi ketika lapisan lambung mengalami iritasi, peradangan\u0026lt;/span\u0026gt; Penyakit Asam Lambung , yaitu kondisi ketika asam lambung naik ke kerongkongan Tukak lambung , yaitu \u0026lt;span style=\color: #000000; font-family: museosans300, sans-serif;\\u0026gt;luka yang muncul pada dinding lambung akibat terkikisnya lapisan dinding lambung. Luka ini juga berpotensi muncul pada dinding bagian pertama usus kecil (duodenum) serta kerongkongan (esofagus)\u0026lt;/span\u0026gt; Irritable Bowel Syndrome , yaitu gangguan pada usus besar yang dapat dipicu oleh stres atau makanan tertentu Dsbnya Alangkah baiknya untuk menelusuri penyebab yang mendasari keluhan, Kamu memeriksakan diri ke Dokter...')

'Pagi Jhunny,Terima kasih atas pertanyaannya.Keluhan berupa sakit pada perut yang dirasakan setiap makan tentunya dapat terasa mengganggu. Hal ini dapat disebabkan oleh hal berikut: Gastritis , yaitu kondisi ketika lapisan lambung mengalami iritasi, peradangan Penyakit Asam Lambung , yaitu kondisi ketika asam lambung naik ke kerongkongan Tukak lambung , yaitu luka yang muncul pada dinding lambung akibat terkikisnya lapisan dinding lambung. Luka ini juga berpotensi muncul pada dinding bagian pertama usus kecil (duodenum) serta kerongkongan (esofagus) Irritable Bowel Syn , yaitu gangguan pada usus besar yang dapat dipicu oleh stres atau makanan tertentu Dsbnya Alangkah baiknya untuk menelusuri penyebab yang mendasari keluhan, Kamu memeriksakan diri ke Dokter...'

In [ ]:
sample_1 = df['answer'].sample(1)

sample_1.values[0]

'Halo,Terimakasih atas pertanyaan yang Anda ajukan.Berdasarkan informasi yang Anda sebutkan, memang kemungkinan Anda mengalami suatu gejala  sembelit atau konstipasi . Konstipasi memang menjadi salah satu faktor terjadinya  ambeien/wasir . Oleh karena itu penanganan ambeien akan tuntas jika konstipasi bisa teratasi.Perlu Anda ketahui bahwa faktor yang mempengaruhi terjadinya konstipasi ada banyak. Selain pola makan yang tidak sehat/kurang mengonsumsi makanan berserat, kondisi psikis, akibat mengonsumsi obat-obatan tertentu, serta gangguan pada kondisi saluran pencernaan bawah juga menjadi pemicu. Tentu hal ini harus ditelusuri lebih lanjut.Diskusi lain terkait konstipasi dan ambeien yang serupa, bisa Anda baca  disini. Jika Anda sudah mengalami gejala ini cukup lama, disarankan untuk memeriksa kembali ke dokter terdekat. Anda bisa memeriksa ke dokter spesialis penyakit dalam agar mendapatkan pemeriksaan yang lebih lengkap, sehingga pengobatan dapat diberikan sesuai kondisi.Anjuran yang

In [ ]:
bersihkan_sapaan(sample_1.values[0])

'tinja pada bayi yang sudah mendapakan makanan pendampin ASI (MP-ASI) bisa berbentuk cairan dengan danya gumpalan-gumapaln dengan warna kuning, hijau, hingga coklat, apabila terlalu cair/encer tidak selalu menggambarkan adanya penyakit atau sesuatu yang berbahaya, karena memang dominasi asupan yang didapatkan oleh bayi lebih banyak adalah berbentuk cairan. Yang perlu diwaspdai dari bentuk dan warna tinja adalah apabila berubah menjadi benar-benar cair menandakan bahwa bayi mengalami diare, atau warnanya berubah menjadi merah, putih, atau hitam. Warna putih terjadi apabila adanya gangguan pada empedu bayi, sedangkan warna hitam akibat adanya pendarahan pada saluran pencernaan, sedangkan warna merah menandakan adanya pendarahan pada bagia usus dekat dubur. Adanya kelainan gangguan pada pencernaan biasanya disertai juga gejala lain seperti rewel, muntah, demam, lemas, bibir menjadi tampak kering, bahkan saat menangis tidak adalagi air mata yang keluar. Nah kalau sudah gejala mencurigakaan

In [ ]:
df['question']

In [ ]:
df['answer-clean'] = df['answer'].apply(bersihkan_sapaan)

In [ ]:
df.head()

,title,question,answer,topics,topic_set,pola 1,pola 2,answer-clean
0,Penyebab dan cara mengatasi BAB berdarah,"Alo dok. Barusan saya BAB, tapi BAB nya keluar darah lumayan banyak. kejadian ini sudah terjadi sekitar 3-4 kalian. setiap saya BAB, pasti selalu ada darah, tapi sebelumnya hanya 1 tetes saja darahnya. sekarang darahnya banyak, warnanya merah gelap (seperti darah haid). sejauh ini saya tidak merasa demam, diare, ataupun gejala lainnya. BAB saya juga terbilang cukup normal. kira kira itu kenapa ya dok? terimakasih","Alo Sania, terima kasih ya sudah bertanya di Alodokter. Keluhan yang anda alami dapat saja merupakan gejala dari : Disentri (bakteri ataupun amuba) Wasir Radang usus Luka pada anus Untuk membantu menentukan apa kemungkinan penyebab keluhan anda, ada baiknya anda juga memerhatikan apakah terdapat benjolan yang keluar dari anus, apakah darah bercampur dengan feses, dan terasa nyeri pada saat atau setelah BAB. Apabila keluhan saat ini masih berulang dan memang banyak, ada baiknya anda memeriksakan keluhan secara langsung ke dokter untuk dilakukan evaluasi dan penanganan pertama. Anda tidak disarankan mengonsumsi obat-obatan apapun tanpa adanya anjuran yang jelas dari dokter anda, terutama obat-obatan untuk menghentikan perdarahan. Anda juga disarankan untuk menghindari mengejan terlalu sering, meningkatkan konsumsi sayur dan buah, memastikan makanan yang anda konsumsi bersih dan matang sempurna serta menjaga asupan cairan minimal 2 liter perharinya. Apabila muncul keluhan seperti pusi...",wasir gangguan-pencernaan,gangguan-pencernaan,1,0,"terima kasih ya sudah bertanya . Keluhan yang anda alami dapat saja merupakan gejala dari : Disentri (bakteri ataupun amuba) Wasir Radang usus Luka pada anus Untuk membantu menentukan apa kemungkinan penyebab keluhan anda, ada baiknya anda juga memerhatikan apakah terdapat benjolan yang keluar dari anus, apakah darah bercampur dengan feses, dan terasa nyeri pada saat atau setelah BAB. Apabila keluhan saat ini masih berulang dan memang banyak, ada baiknya anda memeriksakan keluhan secara langsung ke dokter untuk dilakukan evaluasi dan penanganan pertama. Anda tidak disarankan mengonsumsi obat-obatan apapun tanpa adanya anjuran yang jelas dari dokter anda, terutama obat-obatan untuk menghentikan perdarahan. Anda juga disarankan untuk menghindari mengejan terlalu sering, meningkatkan konsumsi sayur dan buah, memastikan makanan yang anda konsumsi bersih dan matang sempurna serta menjaga asupan cairan minimal 2 liter perharinya. Apabila muncul keluhan seperti pusing, lemas, perdarahan y..."
1,Susah BAB setelah operasi wasir 3 bulan lalu,"Dok , pasca oprasi hemoroid 3 bulan lalu . Saya tidak pernah merasakan perut mulas ingin BAB .tapj lubang anus saya ada dorongan terus untuk BAB . Jika di ikuti fases yg kluar hanya sedikit tapi sering. Apa yg harus syaa lakukan ?","Alo, Terima kasih atas pertanyaannya. Keluhan sulit BAB dengan riwayat operasi hemorrhoid dapat disebabkan oleh banyak hal. Hal ini tergantung sejak kapan keluhan ini dirasakan, gejala penyerta (nyeri perut, nyeri pada anus, BAB keluar darah, diare, dll), riwayat pengobatan, dan lainnya. Terdapat beberapa kemungkinan penyebab dari keluhan yang Anda rasakan, seperti konstipasi/obstipasi, komplikasi pasca operasi, inkontinensia alvi, maupun irritable bowel syndrome . Apabila keluhan sulit BAB ini sudah terjadi setidaknya dalam 2 minggu terakhir ini, sebaiknya periksakan diri ke dokter. Dokter akan mengevaluasi keluhan Anda lebih lengkap lagi, melakukan pemeriksaan fisik, dan pemeriksaan penunjang yang relevan sehingga dokter dapat menentukan penyebab pasti dari keluhan Anda dan memberikan terapi yang sesuai. Untuk sementara waktu, berikut beberapa anjuran yang dapat Anda lakukan, seperti: Mencukupi kebutuhan nutrisi yang sehat dan bergizi. Mengonsumsi makanan yang kaya akan serat, ...",konstipasi wasir,wasir,1,0,"Terima kasih atas pertanyaannya. Keluhan sulit BAB dengan riwayat operasi hemorrhoid dapat disebabkan oleh ba

In [ ]:
df['Clean-Question'] = df['question'].apply(bersihkan_sapaan)

### Preprocessing Word Elongation

In [ ]:
from indoNLP.preprocessing import replace_word_elongation
df['final Answer'] = df['answer-clean'].apply(replace_word_elongation)
df['final Question'] = df['Clean-Question'].apply(replace_word_elongation)

### Preprocessing Replace Slang

In [ ]:
from indoNLP.preprocessing import replace_slang
df['final Answer'] = df['final Answer'].apply(replace_slang)
df['final Question'] = df['final Question'].apply(replace_slang)

In [ ]:
df['final Question last'] = df['title'] + ". "  + df['final Question']

In [ ]:
df.sample(2)

,title,question,answer,topics,topic_set,pola 1,pola 2,answer-clean,Clean-Question,final Answer,final Question,final Question last
20694,sakit kepala bagian belakang disertai jantung berdebar-debar dan sakit di ulu hati,"Assalammu'alaikum,dok saya mau tanya ini udah hampir 1 bulan ini saya mengalami penyakit GERD.. yg saya alami selama ini kepala belakang sering pusing kadang kadang Suka ke ubun2 jantung sering berdebar kencang sakit di ulu hati,sesak napas, sakit tenggorokan,telapak kaki,tangan dingin dok,,, setiap hujan turun penyakit saya ini kambuh lagi yg saya mau tanyakan apa saya kena asam lambung apa penyakit lain ya dok mohon jawab dok,,, thanks donk,,,","Hai.Dari keluhan yang Anda alami, rasa sesak nafas disertai dengan rasa nyeri ulu hati dan sakit kepala mungkin dapat disebabkan karena suatu GERD. GERD atau biasa disebut sebagai penyakit asam lambung adalah suatu keadaan dimana terjadinya kenaikan dari asam lambung ke tenggorokan. Pada orang yang sudah lama mengalami GERD, maka dapat terjadi beberapa komplikasi seperti sering pusing, serta pada keadaan lanjut dapat terjadi erosi tenggorokan sehingga dapat menyebabkan muntah darah. GERD dapat menyebabkan gejala seperti : Nyeri pada dada Dada terasa panas Sesak Nyeri ulu hati Mual muntah Lemas dll Tetapi mungkin juga keadaan yang Anda rasakan dapat disebabkan karena penyebab lain seperti: Asma Infeksi paru Anemia/ Kurang sel darah merah Sakit kepala tipe tegang/ Tension type headache Serangan jantung dll Oleh karena itu, untuk memastikan penyebab dari keluhan Anda, dirasankan agar Anda memeriksakan diri secara langsung ke dokter sehingga dapat dilakukan pe...",asam-lambung,asam-lambung,1,0,"Dari keluhan yang Anda alami, rasa sesak nafas disertai dengan rasa nyeri ulu hati dan sakit kepala mungkin dapat disebabkan karena suatu GERD. GERD atau biasa disebut sebagai penyakit asam lambung adalah suatu keadaan dimana terjadinya kenaikan dari asam lambung ke tenggorokan. Pada orang yang sudah lama mengalami GERD, maka dapat terjadi beberapa komplikasi seperti sering pusing, serta pada keadaan lanjut dapat terjadi erosi tenggorokan sehingga dapat menyebabkan muntah darah. GERD dapat menyebabkan gejala seperti : Nyeri pada dada Dada terasa panas Sesak Nyeri ulu hati Mual muntah Lemas dll Tetapi mungkin juga keadaan yang Anda rasakan dapat disebabkan karena penyebab lain seperti: Asma Infeksi paru Anemia/ Kurang sel darah merah Sakit kepala tipe tegang/ Tension type headache Serangan jantung dll Oleh karena itu, untuk memastikan penyebab dari keluhan Anda, dirasankan agar Anda memeriksakan diri secara langsung ke dokter sehingga dapat dilakukan pemeriksaan lebih lanjut. Mungki...","Assalammu'alaikum,dok saya mau tanya ini udah hampir 1 bulan ini saya mengalami penyakit GERD.. yg saya alami selama ini kepala belakang sering pusing kadang kadang Suka ke ubun2 jantung sering berdebar kencang sakit di ulu hati,sesak napas, sakit tenggorokan,telapak kaki,tangan dingin dok,,, setiap hujan turun penyakit saya ini kambuh lagi yg saya mau tanyakan apa saya kena asam lambung apa penyakit lain ya dok mohon jawab dok,,, thanks donk,,,","Dari keluhan yang Anda alami, rasa sesak nafas disertai dengan rasa nyeri ulu hati dan sakit kepala mungkin dapat disebabkan karena suatu GERD. GERD atau biasa disebut sebagai penyakit asam lambung adalah suatu keadaan dimana terjadinya kenaikan dari asam lambung ke tenggorokan. Pada orang yang sudah lama mengalami GERD, maka dapat terjadi beberapa komplikasi seperti sering pusing, serta pada keadaan lanjut dapat terjadi erosi tenggorokan sehingga dapat menyebabkan muntah darah. GERD dapat menyebabkan gejala seperti : Nyeri pada dada Dada terasa panas Sesak Nyeri ulu hati Mual muntah Lemas dulu Tetapi mungkin juga keadaan yang Anda rasakan dapat disebabkan karena penyebab lain seperti: Asma Infeksi paru Anemia/ Kurang sel darah merah Sakit kepala tipe tegang/ Tension type headache Serangan jantung dulu Oleh karena itu, untuk memastikan penyebab dari keluha

In [ ]:
df[['final Question last', 'final Answer']].to_csv('final.csv', index=False)

In [ ]:
 !curl -F "reqtype=fileupload" -F "fileToUpload=@final.csv" https://catbox.moe/user/api.php

https://files.catbox.moe/q4yhww.csv

In [ ]:
df.isnull().sum()

,0
title,0
question,0
answer,0
topics,0
topic_set,0
pola 1,0
pola 2,0
answer-clean,0
Clean-Question,0
final Answer,0


In [ ]:
df[df['final Answer'] == ""]

,title,question,answer,topics,topic_set,pola 1,pola 2,answer-clean,Clean-Question,final Answer,final Question,final Question last
10260,Obat untuk muntah disertai BAB cair,mlm dok oya dok da beberapa hari ini saya muntah2 dan mencret2 mau tanya dok itu gejalah apa yah dok n apa obatnya....makasi dok sebelumnya...,"Halo,",muntah diare,diare,1,0,,mlm dok oya dok da beberapa hari ini saya muntah2 dan mencret2 mau tanya dok itu gejalah apa yah dok n apa obatnya....makasi dok sebelumnya...,,malam dok oya dok dah beberapa hari ini saya muntah2 dan mencret2 mau tanya dok itu gejalah apa ya dok n apa obatnya....terima kasih dok sebelumnya...,Obat untuk muntah disertai BAB cair. malam dok oya dok dah beberapa hari ini saya muntah2 dan mencret2 mau tanya dok itu gejalah apa ya dok n apa obatnya....terima kasih dok sebelumnya...
20409,Solusi atasi nyeri pada punggung,Saya kalo abis makan mie punggung belikat nyeri banget gimana ya solusi nya dok,Hai Restu Yani Handayani,asam-lambung,asam-lambung,0,0,,Saya kalo abis makan mie punggung belikat nyeri banget gimana ya solusi nya dok,,Saya kalo habis makan mie punggung belikat nyeri banget bagaimana ya solusi nya dok,Solusi atasi nyeri pada punggung. Saya kalo habis makan mie punggung belikat nyeri banget bagaimana ya solusi nya dok


In [ ]:
df.to_csv('final-[1].csv', index=False)

In [ ]:
!curl -F "reqtype=fileupload" -F "fileToUpload=@final-[1].csv" https://catbox.moe/user/api.php

https://files.catbox.moe/bu8lgd.csv

# Training 🏋️‍♀️

Untuk training model model LLM kami menggunakan salah satu teknik fine tuninng yaitu PEFT (Parameter Efficient Fine Tune) yaitu QLoRA

- QLoRA adalah teknik fine-tuning model bahasa besar (LLM) yang mengurangi penggunaan memori secara signifikan dibandingkan dengan fine-tuning penuh. Ini dilakukan dengan mengkuantisasi bobot model menjadi 4-bit dan menggunakan Low-Rank Adapters (LoRA) untuk melatih hanya sebagian kecil dari parameter model.

- Unsloth adalah library yang mengoptimalkan proses QLoRA, membuatnya 2x lebih cepat dibandingkan implementasi standar, terutama pada GPU seperti T4, V100, dan Ampere+.

In [ ]:
torch.cuda.is_available()

True

### Berikut adalah model model yang kami Gunakan

In [ ]:
import torch
from unsloth import FastLanguageModel

In [ ]:
Models = [
  "unsloth/mistral-7b-v0.3",
  "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
  "unsloth/llama-3-8b-bnb-4bit",
  "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct",
  "GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct",
  "Yellow-AI-NLP/komodo-7b-base",
]

## Load Model

Pemilihian Modal , Sequence Length dan `load in 4 bit`

- Penggunaan parameter `load_in_4bit` = True pada bagian inisialisasi model dengan FastLanguageModel.from_pretrained adalah tanda utama bahwa proses ini menggunakan teknik QLoRA.

- Seperti yang sudah dijelaskan sebelumnya, QLoRA adalah variasi dari LoRA yang mengkuantisasi model dasar (base model) menjadi 4-bit sebelum menambahkan adapter LoRA. Parameter load_in_4bit=True secara spesifik menginstruksikan library  untuk memuat model dalam format 4-bit terkuantisasi tersebut.

In [ ]:
# Konfigurasi
selected_model = "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct" # @param ["unsloth/mistral-7b-v0.3", "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", "unsloth/llama-3-8b-bnb-4bit", "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct", "GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct", "Yellow-AI-NLP/komodo-7b-base"]
max_seq_length = 2048  # @param {type:"slider", min:512, max:8192, step:256}
load_in_4bit = True  # @param {type:"boolean"}
dtype = None  # @param ["None", "float16", "bfloat16", "float32"] {type:"raw"}
HuggingFace_token = "hf_..."  # @param {type:"string"}

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = selected_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = HuggingFace_token,
)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.096 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Penerapan LoRA

`FastLanguageModel.get_peft_model` dari Unsloth untuk menerapkan **fine-tuning efisien dengan LoRA (Low-Rank Adaptation)** pada model LLM.

* `model`: Model dasar 4-bit yang akan ditambahkan adapter LoRA.
* `r=16`: Rank LoRA adapter, menentukan kapasitas adaptasi.
* `target_modules`: Layer transformer tempat LoRA disuntikkan (proyeksi query, key, value, output, dan FFN).
* `lora_alpha=16`: Faktor penskalaan LoRA.
* `lora_dropout=0`: Tidak ada dropout, dioptimalkan oleh Unsloth.
* `bias="none"`: Tidak menambahkan bias, untuk efisiensi.
* `use_gradient_checkpointing="unsloth"`: Hemat memori saat training input panjang.
* `random_state=3407`: Seed untuk reproduktibilitas.
* `use_rslora=False`, `loftq_config=None`: Tidak menggunakan varian tambahan LoRA/quant.



In [ ]:
r = 16  # @param {type:"slider", min:4, max:128, step:4}
lora_alpha = 16  # @param {type:"slider", min:8, max:128, step:8}
lora_dropout = 0.0  # @param {type:"slider", min:0.0, max:0.5, step:0.05}
bias = "none"  # @param ["none", "all", "lora_only"]
use_gradient_checkpointing = "unsloth"  # @param ["unsloth", "torch", "none"]
random_state = 3407  # @param {type:"number"}
use_rslora = False  # @param {type:"boolean"}
loftq_config = None  # @param ["None"] {type:"raw"}

# === Apply LoRA ===
model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = use_rslora,
    loftq_config = None if loftq_config == "None" else loftq_config,
)


## Data preparation sebelum Training


### Downloa Data Train

Data hasil dari preprocessing sebelumnya kami simpan dalam platform Hugging Face agar mempermudah dalam dowload Dataset

In [ ]:
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian", split = "train")

### Data Formatting

Pada tahap ini, dilakukan proses **formatting data** untuk pelatihan model bahasa berbasis instruksi. Dataset yang berisi pasangan *input* dan *output* diubah ke dalam format **Alpaca-style prompt** yang terdiri dari instruksi, konteks, dan respons. Tujuannya adalah agar model dapat belajar merespons pertanyaan medis dalam bentuk dialog yang konsisten dan mudah dipahami.

Langkah-langkah proses:

1. **Template Prompt:**
   Dibuat template teks (`alpaca_prompt`) dengan struktur sebagai berikut:

   * **Instruction:** Petunjuk untuk menjawab pertanyaan medis dengan jelas dan informatif.
   * **Input:** Konteks atau pertanyaan dari pengguna.
   * **Response:** Jawaban yang diharapkan dari model.

2. **Penambahan EOS Token:**
   Ditambahkan `EOS_TOKEN` (End-of-Sequence) pada akhir setiap prompt agar model mengetahui batas akhir output saat pelatihan maupun inferensi.

3. **Fungsi Format Prompt:**
   Fungsi `formatting_prompts_func` menggabungkan input dan output dari dataset ke dalam template `alpaca_prompt`, lalu menghasilkan list teks terformat dalam kolom `"text"`.

4. **Penerapan ke Dataset:**
   Fungsi ini diaplikasikan ke seluruh dataset menggunakan `map()` dari `datasets`, menghasilkan dataset yang siap digunakan untuk proses fine-tuning dengan format instruksi yang konsisten.

Hasil akhirnya adalah dataset dalam bentuk teks terstruktur yang sesuai dengan format pelatihan model instruksi seperti Alpaca dan LoRA, sehingga model dapat belajar memberi respons dengan gaya dialog (conversation).


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input_text, output_text in zip(inputs, outputs):
        text = alpaca_prompt.format(input_text, output_text) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }
pass

from datasets import load_dataset
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|███████████████████████████████████████████████████████████████████████████| 19845/19845 [00:00<00:00, 42664.76 examples/s]


## Konfigurasi Training


Pada bagian ini, dilakukan inisialisasi dan konfigurasi `trainer` untuk fine-tuning model LLM  menggunakan `SFTTrainer` dari library trl.

Di sini, ditentukan model yang akan dipilih unutk pelatihan (model), tokenizer yang dipakai , data yang akan digunakan untuk training , serta berbagai parameter pelatihan yang mendetail melalui objek TrainingArguments.

 Pengaturan ini mencakup hal-hal seperti ukuran data yang diproses per langkah pelatihan `(steps)` `(per_device_train_batch_size dan gradient_accumulation_steps)`, warmup_steps, Jumlah epoch dalam pelatihan `(num_train_epochs)`, `learning_rate`,  optimizer yang digunakan `(optim)`.

Singkatnya, pada bagian ini adalah tahapan penyiapan lengkap enviroment dan pengaturan yang akan digunakan untuk menjalankan proses fine-tuning model pada data yang telah disiapkan sebelumnya.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

per_device_train_batch_size = 8  # @param {type:"slider", min:1, max:64, step:1}
gradient_accumulation_steps = 4  # @param {type:"slider", min:1, max:32, step:1}
num_train_epochs = 1  # @param {type:"slider", min:1, max:10, step:1}
learning_rate = 2e-4  # @param {type:"number"}
warmup_steps = 5  # @param {type:"number"}
logging_steps = 1  # @param {type:"number"}
weight_decay = 0.01  # @param {type:"number"}
lr_scheduler_type = "linear"  # @param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]
optim = "adamw_8bit"  # @param ["adamw_8bit", "adamw_torch", "adamw_hf", "adafactor"]
packing = False  # @param {type:"boolean"}
output_dir = "outputs"  # @param {type:"string"}
report_to = "wandb"  # @param ["wandb", "tensorboard", "none"] {type:"raw"}
seed = 3407  # @param {type:"number"}

# === Trainer Setup ===
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = packing,
    args = TrainingArguments(
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = warmup_steps,
        num_train_epochs = num_train_epochs,
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = logging_steps,
        optim = optim,
        weight_decay = weight_decay,
        lr_scheduler_type = lr_scheduler_type,
        seed = seed,
        output_dir = output_dir,
        report_to = report_to,
        # project = "FP-NLP"
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/19845 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 80GB HBM3. Max memory = 79.096 GB.
7.625 GB of memory reserved.


## Memulai Training

Pada bagian ini, dilakukan eksekusi proses fine-tuning model.

Kode  **trainer.train()** memanggil metode **train()** dari objek trainer yang sudah dikonfigurasi sebelumnya. Ini akan  memulai training LLM menggunakan dataset dan parameter yang telah ditentukan dalam objek TrainingArguments sebelumnya


Selama proses ini, model akan belajar dari data pelatihan, menyesuaikan bobotnya melalui backpropagation dan optimisasi untuk meningkatkan kemampuannya dalam menjawab pertanyaan medis sesuai dengan format instruksi yang diinginkan.

Output dari proses pelatihan, seperti loss dan metrik lainnya, akan disimpan dalam variabel trainer_stats.


Setelah ini selesai, model Anda akan menjadi model yang sudah di-fine-tuning dan siap untuk digunakan dalam inferensi (menjawab pertanyaan baru).

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,845 | Num Epochs = 1 | Total steps = 620
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.948400
2,2.062900
3,2.027800
4,1.961200
5,1.951000
6,1.818100
7,1.741500
8,1.631500
9,1.637800
10,1.593300


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1877.8087 seconds used for training.
31.3 minutes used for training.
Peak reserved memory = 9.266 GB.
Peak reserved memory for training = 1.641 GB.
Peak reserved memory % of max memory = 11.715 %.
Peak reserved memory for training % of max memory = 2.075 %.


<a name="Inference"></a>
# Inference 🧠
Percobaan Infernce Setelah Training Model

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "saya mengalami sakit perut?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
saya mengalami sakit perut?

### Response:
dari keadaan yang anda alami, rasa sakit pada perut bagian atas dapat disebabkan karena gangguan kesehatan yang terkait organ lambung. berikut ini beberapa penyakit yang dapat menyerang organ lambung, diantaranya adalah : gastritis=== peradangan pada dinding lambung bagian dalam, penyakit asam lambung / gerd, tukak peptik=== luka yang terjadi pada dinding lambung bagian dalam, kanker lambung. selain dari keadaan yang telah disebutkan diatas, rasa


In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Saya mengalami sakit di anus?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nJawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\nSaya mengalami sakit di anus?\n\n### Response:\nsaya akan membantu menjawab pertanyaan anda. berdasarkan gejala yang anda alami, kemungkinan besar anda mengalami wasir atau ambeien. wasir merupakan kondisi dimana terjadi pembengkakan pada pembuluh darah vena di sekitar anus. gejala yang umumnya muncul pada wasir adalah: gatal-gatal di sekitar anus, terutama pada malam hari nyeri pada anus muncul benjolan pada anus yang berdarah bila disentuh atau saat buang air besar muncul lendir pada saat buang air besar terdapat rasa tidak nyaman di sekitar anus namun, ada beberapa kemungkinan penyebab lainnya yang menyebabkan gejala yan

<a name="Save"></a>
# Saving, loading finetuned models 📩
Untuk menyimpan model final sebagai adapter LoRA, menggunakan `push_to_hub` Huggingface untuk penyimpanan online atau save_pretrained untuk penyimpanan lokal.

[CATATAN] Ini HANYA menyimpan adapter LoRA, bukan model penuh. Untuk menyimpan ke format 16-bit atau GGUF

## menyimpan Adapter model LoRA di Hugging Face atau local

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

In [ ]:
model.push_to_hub("farwew/lora_model", token = "hf_REMOVED_BY_SCRIPT") # Online saving

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.07it/s]


Saved model to https://huggingface.co/farwew/lora_model


In [ ]:
! zip -r lora_model.zip lora_model

  adding: lora_model/ (stored 0%)
  adding: lora_model/README.md (deflated 43%)
  adding: lora_model/adapter_model.safetensors (deflated 7%)
  adding: lora_model/adapter_config.json (deflated 56%)


In [ ]:
!curl -F "reqtype=fileupload" -F "fileToUpload=@lora_model.zip" https://catbox.moe/user/api.php

https://files.catbox.moe/d95pr5.zip

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>

# Evaluasi Metrik Kunatitatif  ( Bert Score, Blue, Rouge )


## Pada bagian ini, kami melakukan evaluasi kuantitatif terhadap performa **Base (sebelum di Fine Tune)** dan  model yang telah di-fine-tuning
.

Evaluasi ini menggunakan metrik standar dalam penilaian kualitas teks yang dihasilkan oleh model bahasa, yaitu **BERT Score, BLEU, dan ROUGE**. Tujuannya adalah untuk mengukur seberapa dekat atau relevan teks jawaban yang dihasilkan model dibandingkan dengan jawaban referensi dari dataset.

*   **BERT Score**: Mengukur kesamaan semantik antara teks yang dihasilkan dan teks referensi dengan memanfaatkan representasi kontekstual dari model BERT.
*   **BLEU (Bilingual Evaluation Understudy)**: Mengukur presisi n-gram, yaitu seberapa banyak urutan kata dalam teks yang dihasilkan muncul dalam teks referensi. Umumnya digunakan untuk menilai terjemahan mesin, namun juga relevan untuk tugas generasi teks lainnya.
*   **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**: Mengukur *recall* n-gram, yaitu seberapa banyak urutan kata dalam teks referensi yang muncul dalam teks yang dihasilkan. Umumnya digunakan untuk menilai ringkasan teks.

Dengan menggunakan metrik-metrik ini, kami dapat memperoleh gambaran objektif mengenai kualitas teks yang dihasilkan oleh model kami sebelum dan setelah fine-tuning.

Load LoRA model dan adapter

## Download Dependencies

In [ ]:
%%capture
! pip install unsloth
! pip install rouge_score
! pip install evaluate

In [ ]:
%%capture
! pip install -U peft
! pip install bert_score

## Import Dependencies

In [ ]:
from tqdm import tqdm
import kagglehub
import os

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datasets import load_dataset
import evaluate
import torch
import bert_score
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import os
import re

In [ ]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [ ]:
from peft import PeftModel

## Load dataset untuk Testing (Evaluation)

In [ ]:
# 1. Load dataset
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian")

In [ ]:
# 3. Ambil data test
dataset_eval = dataset["test"]

# 4. Konversi ke pandas DataFrame
df = dataset_eval.to_pandas()
df.head()

,instruction,input,output,topic_set,topics
0,Selalu tersedak setiap makan dan minum,Selalu tersedak setiap makan dan minum. Dok sa...,batuk adalah respon tubuh untuk mengeluarkan b...,gastritis,penyakit-parkinson multiple-sclerosis gastritis
1,Apa obat untuk demam?,Apa obat untuk demam?. malam dok saya mau tany...,"kepala pusing, perut mual, punggung sakit dan ...",mual,flu pilek sakit-kepala nyeri-punggung mual
2,Cara mengatasi perut kembung pada anak,Cara mengatasi perut kembung pada anak. punya ...,perut kembung menandakan adanya penumpukan gas...,perut-kembung,anak gastritis perut-kembung
3,Perut Bagian Bawah Kembung,"Perut Bagian Bawah Kembung. saya mau bertanya,...","seperti: sayur: kol, brokoli, kembang kol, asp...",gangguan-pencernaan,gangguan-pencernaan
4,Nyeri dada menjalar ke punggung,Nyeri dada menjalar ke punggung. Malam dok tad...,saya coba bantu jawab. keluhan nyeri dada menj...,sakit-maag,nyeri-dada sakit-maag


## Stratify  Sampling for Testing

Kami melakukan stratified sampling terhadap topik pertanyaan untuk memilih 100 sampel acak dari data uji. Sampling ini diambil dari 1.000 data uji sebelumnya dengan tujuan untuk mengurangi beban komputasi mengingat keterbatasan perangkat yang digunakan.

In [ ]:
# 5. Stratified split berdasarkan kolom 'topic_set'
train_df, stratified_sample_df = train_test_split(
    df,
    test_size=100,  # ubah sesuai proporsi yang diinginkan
    stratify=df["topic_set"],
    random_state=42
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
stratified_dataset = Dataset.from_pandas(stratified_sample_df)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

##   Evaluasi Base Model (Bukan Fine Tune)

Pemilihan Model sesuai dengan Model Base yang mau dievaluasi

In [ ]:
# Konfigurasi
selected_model = "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct" # @param ["unsloth/mistral-7b-v0.3", "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", "unsloth/llama-3-8b-bnb-4bit", "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct", "GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct", "Yellow-AI-NLP/komodo-7b-base"]
max_seq_length = 2048  # @param {type:"slider", min:512, max:8192, step:256}
load_in_4bit = True  # @param {type:"boolean"}
dtype = None  # @param ["None", "float16", "bfloat16", "float32"] {type:"raw"}
HuggingFace_token = "hf_..."  # @param {type:"string"}

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = selected_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = HuggingFace_token,
)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.096 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Formatting data

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
{}

### Response:
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input_text, output_text in zip(inputs, outputs):
        text = alpaca_prompt.format(input_text, output_text) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }
pass

### Mulai Inference Base Model (sebelum Fine Tune) untuk Evaluasi

In [ ]:
# # === Evaluasi === #
# bleu = evaluate.load("bleu")
# rouge = evaluate.load("rouge")
generated_outputs = []
reference_outputs = []
all_instructions = []
all_inputs = []

In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"\p{P}+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

### Inference
Bagian kode ini melakukan inferensi (menghasilkan jawaban) menggunakan model dasar untuk setiap pertanyaan di dataset uji, kemudian mengumpulkan jawaban yang dihasilkan bersama dengan jawaban referensi untuk keperluan evaluasi metrik di tahap berikutnya. Beberapa contoh jawaban juga ditampilkan.



In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    # Gunakan prompt yang hanya menerima input (karena instruksi tetap)
    prompt = alpaca_prompt.format(example["input"], "")  # output dikosongkan untuk inferensi

    # Tokenisasi dan pindahkan ke GPU
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Inference (tanpa perhitungan gradien)
    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    # Decode output dan ekstrak bagian response-nya
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    # Cetak sampel jika masih kurang dari 10
    if num < 10:
        print(prompt)
        print()
        print('response:', response)
        print('-' * 80)

    # Simpan hasil untuk evaluasi
    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append("Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.")  # atau bisa juga diabaikan jika instruksi selalu sama
    all_inputs.append(example["input"])
    num += 1

Evaluating Base Model:   1%|          | 1/100 [01:06<1:49:34, 66.41s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan ma

Evaluating Base Model:   2%|▏         | 2/100 [02:09<1:44:58, 64.27s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Response:


response: Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
solusi untuk mengat
--------------------------------------------------------------------------------


Evaluating Base Model:   3%|▎         | 3/100 [02:42<1:21:05, 50.16s/it]

...

Evaluating Base Model:   8%|▊         | 8/100 [06:02<56:41, 36.97s/it]  

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Response:


response: saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari sel

Evaluating Base Model:   9%|▉         | 9/100 [06:14<44:24, 29.29s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.

### Input:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Response:


response: Saya telah mempelajari dokumen medis Anda. Terima kasih telah memilih kami untuk melayani kebutuhan kesehatan Anda. Saya memiliki beber

Evaluating Base Model: 100%|██████████| 100/100 [53:14<00:00, 31.95s/it]


### Menyimpan Hasil Output dari Base Modal

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
})

df.to_csv(f"base_model_outputs.csv", index=False)

In [ ]:
base = pd.read_csv('base_model_outputs.csv')
base.sample(1)

In [ ]:
base.fillna('empty', inplace=True)

### Evaluasi Kunatitatif untuk Base Model

In [ ]:
# Buang baris yang memiliki nan pada kolom generated atau reference
df_clean = base.dropna(subset=['generated', 'reference']).copy()

preds = df_clean['generated'].tolist()
refs = df_clean['reference'].tolist()

# Pastikan references berbentuk list of list untuk BLEU
refs_for_bleu = [[ref] for ref in refs]

bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
rouge_scores = compute_rouge_per_row(preds, refs)

# Buat dict untuk dataframe baru
data_new = {}

for n in range(1, 5):
    data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

In [ ]:
bertscore_eval_scores = compute_bertscore_per_row(preds, refs_for_bleu)

In [ ]:
df_bert_score = pd.DataFrame(bertscore_eval_scores)
df_bert_score

,prediction,reference,bert_precision,bert_recall,bert_f1
0,"Maaf, dokter tidak tersedia pada hari ini. Berikut ini adalah informasi untuk menjawab pertanyaan anda.\n\nMaag adalah penyakit yang terjadi akibat luka pada perut. Maag terjadi ketika luka di perut mengalami infeksi dan menyebabkan perdarahan. Maag terjadi akibat infeksi virus atau bakteri. Maag juga dapat terjadi akibat kondisi lain seperti kanker, cedera perut, atau infeksi yang disebabkan oleh bakteri.\n\nGejala maag yang sering muncul adalah sakit perut, muntah, dan mual. Gejala-gejala ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.705103,0.661096,0.682391
1,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\nsolusi untuk mengat",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.644507,0.582390,0.611876
2,"Assalamu'alaikum,\n\nPerut sakit, mual dan bersendawa berlebihan. Saya ingin tany dok.. istri saya merasakan sakit perut, kepala pusing di sertai mual2 dan bersendawa yang berlebih.. kira-kira apa penyebab nya dan bagaimana solusi mengatasinya. Terima kasih.\n\nTerima kasih atas pertanyaan medis Anda. Apakah perut sakit Anda berhubungan dengan mual, pusing, dan bersendawa? Jika ya, ini merupakan gejala dari penyakit apapun. Saya akan mencoba menjawab pertanyaan Anda sebaik mungkin dengan inf...","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...",0.634713,0.627972,0.631325
3,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relev","sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala yang anda alami dan melakukan pemeriksaan fisik yang diperlukan. pemeriksaan penunjang seperti tes darah, tes urine, usg mungkin akan dokter anjurkan. terapi yang sesuai akan dokter berikan.anjuran yang dapat anda lakukan: konsumsi makanan bersih dan bergizi secara teratur minum air putih dalam jumlah cukup minimal 10 gelas perhari hindari kebiasaan menahan buang air kecil dan buang air besar manajemen stres s...",0.661312,0.605073,0.631944
4,"Kondisi alergi makanan seafood dapat ditangani dengan beberapa cara, antara lain:\n1. Mencegah kontak dengan bahan alergi. Dengan cara ini, Anda dapat memastikan bahwa tidak ada bahan alergi yang masuk ke tubuh Anda. Hal ini dapat dilakukan dengan cara menghindari makanan yang mengandung bahan alergi.\n2. Mengonsumsi obat anti-inflamasi. Obat anti-inflamasi dapat membantu mengurangi gejala alergi, seperti gatal-gatal dan bengkak. Obat anti-inflamasi yang sering 

In [ ]:
df_scores = pd.DataFrame(data_new)
df_scores = df_scores * 100
df_scores = pd.concat([df_bert_score, df_scores], axis=1)
df_scores.head(2)

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Maaf, dokter tidak tersedia pada hari ini. Berikut ini adalah informasi untuk menjawab pertanyaan anda.\n\nMaag adalah penyakit yang terjadi akibat luka pada perut. Maag terjadi ketika luka di perut mengalami infeksi dan menyebabkan perdarahan. Maag terjadi akibat infeksi virus atau bakteri. Maag juga dapat terjadi akibat kondisi lain seperti kanker, cedera perut, atau infeksi yang disebabkan oleh bakteri.\n\nGejala maag yang sering muncul adalah sakit perut, muntah, dan mual. Gejala-gejala ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.705103,0.661096,0.682391,17.169811,6.495710,2.518932,1.319653,19.973369,3.471295,11.717710,13.581891
1,"Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\nsolusi untuk mengat",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.644507,0.582390,0.611876,0.774941,0.248743,0.172068,0.144252,4.705882,0.000000,4.705882,4.705882


### Contoh Hasil Rata Rata Tiap metrik untuk Base Model

In [ ]:
# Calculate the mean of the numerical columns only
numerical_cols = ['bert_precision', 'bert_recall', 'bert_f1', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'rouge1', 'rouge2', 'rougeL', 'rougeLSum']
mean_scores = df_scores[numerical_cols].mean()

print("\nRata-rata Skor:")
mean_scores


Rata-rata Skor:


,0
bert_precision,0.628678
bert_recall,0.600173
bert_f1,0.613041
bleu1,8.522933
bleu2,2.880937
bleu3,1.473571
bleu4,0.980875
rouge1,12.311867
rouge2,1.299153
rougeL,7.692550


### Menyimpan Hasil Evaluasi

In [ ]:
import os
os.makedirs('Hasil Eval revisi')
mean_scores.to_csv('Hasil Eval revisi/eval_basemodel_4_mean.csv')

In [ ]:
df_scores = pd.concat([base_4[['input']], df_scores], axis=1)
df_scores.head(2)

,input,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","Maaf, dokter tidak tersedia pada hari ini. Berikut ini adalah informasi untuk menjawab pertanyaan anda.\n\nMaag adalah penyakit yang terjadi akibat luka pada perut. Maag terjadi ketika luka di perut mengalami infeksi dan menyebabkan perdarahan. Maag terjadi akibat infeksi virus atau bakteri. Maag juga dapat terjadi akibat kondisi lain seperti kanker, cedera perut, atau infeksi yang disebabkan oleh bakteri.\n\nGejala maag yang sering muncul adalah sakit perut, muntah, dan mual. Gejala-gejala ...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.705103,0.661096,0.682391,17.169811,6.495710,2.518932,1.319653,19.973369,3.471295,11.717710,13.581891
1,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?","Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.\n\n### Input:\nsolusi untuk mengat",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.644507,0.582390,0.611876,0.774941,0.248743,0.172068,0.144252,4.705882,0.000000,4.705882,4.705882


In [ ]:
df_scores.to_csv('Hasil Eval revisi/eval_basemodel_4.csv')

##   Evaluasi Fine Tune Model

Pemilihan Model sesuai dengan Model Fine Tune yang mau dievaluasi

Pada bagian ini, contoh  dilakukan pemuatan adapter LoRA yang sudah di-fine-tuning dari Hugging Face **("farwew/DoctorsAnswerTextDataset-in-IndonesianUnsloth-llama-3-8b-bnb-4bit")** menggunakan `PeftModel.from_pretrained`.

Adapter ini dimuat di atas model dasar (model) yang sebelumnya sudah ada di memori (sudah di load sebelumnya).

Proses ini menggabungkan penyesuaian bobot yang telah dilatih (adapter LoRA) dengan model dasar, sehingga model siap untuk melakukan inferensi dengan performa yang ditingkatkan sesuai hasil fine-tuning.

In [ ]:
model = PeftModel.from_pretrained(model, "farwew/DoctorsAnswerTextDataset-in-IndonesianUnsloth-llama-3-8b-bnb-4bit")

Sama seperti sebelumnay proses nya kita melakukan inferensi dan menyimpan hasil output dari model

In [ ]:
generated_outputs = []
reference_outputs = []
all_inputs = []  # Untuk menyimpan input
all_instructions = []  # Untuk menyimpan instruction

batch_size = 1
MAX_DISPLAY = 10
displayed = 0

# === TIMESTAMP UNTUK NAMA FILE === #
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"evaluation_results_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

### Proses inferensi Fine Tune Model

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    # Gunakan prompt yang hanya menerima input (karena instruksi tetap)
    prompt = alpaca_prompt.format(example["input"], "")  # output dikosongkan untuk inferensi

    # Tokenisasi dan pindahkan ke GPU
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Inference (tanpa perhitungan gradien)
    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    # Decode output dan ekstrak bagian response-nya
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    # Cetak sampel jika masih kurang dari 10
    if num < 10:
        print(prompt)
        print()
        print('response:', response)
        print('-' * 80)

    # Simpan hasil untuk evaluasi
    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append("Jawab pertanyaan medis berikut dengan bahasa yang mudah dipahami. Pastikan respons mencakup: penjelasan kondisi, informasi penting, dan saran yang relevan.")  # atau bisa juga diabaikan jika instruksi selalu sama
    all_inputs.append(example["input"])
    num += 1

### menyimpan hasil output dari model

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
})

df.to_csv(f"fine_tuned_outputs.csv", index=False)

### Evaluasi Kunatitatif untuk Fine Tune  Model

In [ ]:
def evaluate_generation_scores(data):
    # Buang baris yang memiliki NaN pada kolom generated atau reference
    df_clean = data.dropna(subset=['generated', 'reference']).copy()

    preds = df_clean['generated'].tolist()
    refs = df_clean['reference'].tolist()

    # Pastikan references berbentuk list of list untuk BLEU
    refs_for_bleu = [[ref] for ref in refs]

    # Hitung BLEU dan ROUGE per baris
    bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
    rouge_scores = compute_rouge_per_row(preds, refs)

    # Hitung BERTScore per baris
    bert_scores = compute_bertscore_per_row(preds, refs_for_bleu)

    # Siapkan dictionary untuk skor
    data_new = {}

    for n in range(1, 5):
        data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

    data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
    data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
    data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
    data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

    # Tambahkan BERTScore per row
    data_new['bert_precision'] = [score['bert_precision'] for score in bert_scores]
    data_new['bert_recall'] = [score['bert_recall'] for score in bert_scores]
    data_new['bert_f1'] = [score['bert_f1'] for score in bert_scores]

    # Konversi ke DataFrame dan skalakan ke persen
    df_scores = pd.DataFrame(data_new)
    df_scores = df_scores * 100

    # Tambahkan kolom input, reference, dan generated
    df_scores['input'] = df_clean['input'].values
    df_scores['reference'] = df_clean['reference'].values
    df_scores['generated'] = df_clean['generated'].values

    # Hitung rata-rata skor
    mean_scores = df_scores.drop(columns=['input', 'reference', 'generated']).mean()

    return df_scores, mean_scores


In [ ]:
ft_model = pd.read_csv('fine_tuned_outputs.csv')

In [ ]:
df_ft, mean_ft = evaluate_generation_scores(ft_model)

In [ ]:
df_ft.head(2)

,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,bert_precision,bert_recall,bert_f1,input,reference,generated
0,34.997256,16.489537,7.171295,3.347295,34.394904,6.823028,14.012739,14.012739,74.620163,72.101730,73.339337,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","maag atau biasa disebut sebagai gastritis adalah suatu penyakit yang disebabkan oleh adanya peradangan pada dinding lambung. gastritis terjadi karena lapisan lambung mengalami iritasi, peradangan, atau pengikisan oleh asam lambung yang diproduksi oleh sel-sel dalam lambung. gastritis dapat disebabkan oleh beberapa faktor berikut: pola makan yang buruk, seperti terlambat makan, makan makanan yang pedas, asam, berlemak, atau yang menghasilkan banyak gas, seperti kol, kacang-kacangan, minuman b..."
1,54.430380,37.239244,25.657258,17.182931,54.861111,24.475524,43.055556,43.055556,79.042411,78.676975,78.859270,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"konstipasi dapat disebabkan oleh beberapa hal berikut ini: kebiasaan mengabaikan keinginan untuk buang air besar kurang minum kurang konsumsi serat kurang aktif dan jarang berolahraga penurunan berat badan yang mendadak efek samping obat-obatan tertentu penyakit medis tertentu seperti penyakit crohn, penyumbatan usus, diabetes, gangguan tiroid, kanker usus, dsb. untuk memastikan penyebab dari konstipasi yang anda alami, maka sebaiknya anda memeriksakan diri secara langsung ke dokter. dokter ..."


### Contoh hasil rata rata tiap metrik Evaluasi untuk Fine Tune Model

In [ ]:
mean_ft

,0
bleu1,26.152145
bleu2,12.577292
bleu3,6.848691
bleu4,4.149928
rouge1,30.238819
rouge2,6.806361
rougeL,15.668782
rougeLSum,15.668782
bert_precision,69.596678
bert_recall,69.194938


### Menyimpan Hasil Evaluasi

In [ ]:
ft_model_4.to_csv('Hasil Eval revisi/eval_ft_model_4.csv')

In [ ]:
mean_ft.to_csv('Hasil Eval revisi/eval_ft_model_4_mean.csv')

In [ ]:
!zip -r /content/Hasil_Eval_revisi-4.zip /content/Hasil\ Eval\ revisi

  adding: content/Hasil Eval revisi/ (stored 0%)
  adding: content/Hasil Eval revisi/eval_ft_model_4_mean.csv (deflated 40%)
  adding: content/Hasil Eval revisi/eval_basemodel_4_mean.csv (deflated 38%)
  adding: content/Hasil Eval revisi/eval_basemodel_4.csv (deflated 74%)
  adding: content/Hasil Eval revisi/eval_ft_model_4.csv (deflated 73%)


# Evaluasi Menggunakan LLM as Judges 🤖

## <img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

Untuk mengevaluasi kualitas jawaban dari model, kami menggunakan pendekatan **LLM-as-a-Judge**, di mana model bahasa besar (LLM) digunakan untuk menilai kualitas output secara semantik, bukan hanya berdasarkan kemiripan kata. Pendekatan ini memberikan evaluasi yang lebih dekat dengan penilaian manusia, terutama dalam konteks tugas pemahaman bahasa alami yang kompleks seperti tanya jawab medis.

Dalam penelitian ini, kami menggunakan platform **[Opik](https://www.comet.com/site/products/opik/)** dari **Comet**, yaitu framework evaluasi open-source yang mendukung integrasi dengan berbagai metrik penilaian berbasis LLM. Kami memilih Opik karena:

* Mendukung **evaluasi otomatis berbasis semantik**,
* Mampu menghasilkan **skor numerik dan penilaian kualitatif**

### ✅ Metrik Evaluasi yang Digunakan

Berikut adalah empat metrik utama yang digunakan untuk menilai jawaban model:

1. **Hallucination**
   Menilai apakah jawaban mengandung **informasi yang tidak akurat atau tidak berdasarkan input**. Skor tinggi menunjukkan fakta yang sesuai dan tidak mengada-ada (*non-hallucinated*).

2. **Answer Relevance**
   Mengukur sejauh mana jawaban yang diberikan **relevan dan sesuai dengan pertanyaan**. Metrik ini penting untuk mengetahui apakah model memahami konteks dengan benar.

3. **Usefulness**
   Mengevaluasi apakah jawaban yang dihasilkan **memberikan nilai tambah atau membantu pengguna**, terutama dalam konteks praktis atau medis.

4. **Moderation**
   Menilai apakah jawaban **mengandung konten yang berbahaya, tidak pantas, atau melanggar etika**. Skor tinggi menunjukkan bahwa jawaban aman dan layak ditampilkan ke publik.


### 📊 Output Penilaian

Setiap metrik menghasilkan **skor dalam rentang 0–10**, di mana:

* Skor **lebih tinggi** menunjukkan kualitas yang lebih baik (misalnya relevansi tinggi, tidak halu, bermanfaat, dan aman).
* Sistem juga dapat memberikan **penjelasan kualitatif** sebagai justifikasi atas skor yang diberikan.

---



### Contoh promt yang digunakan untuk Evaluasi

#### Hallucination
```
You are an expert judge tasked with evaluating the faithfulness of an AI-generated answer to the given context. Analyze the provided INPUT, CONTEXT, and OUTPUT to determine if the OUTPUT contains any hallucinations or unfaithful information.

Guidelines:

1. The OUTPUT must not introduce new information beyond what's provided in the CONTEXT.
2. The OUTPUT must not contradict any information given in the CONTEXT.
3. The OUTPUT should not contradict well-established facts or general knowledge.
4. Ignore the INPUT when evaluating faithfulness; it's provided for context only.
5. Consider partial hallucinations where some information is correct but other parts are not.
6. Pay close attention to the subject of statements. Ensure that attributes, actions, or dates are correctly associated with the right entities (e.g., a person vs. a TV show they star in).
7. Be vigilant for subtle misattributions or conflations of information, even if the date or other details are correct.
8. Check that the OUTPUT doesn't oversimplify or generalize information in a way that changes its meaning or accuracy.

Analyze the text thoroughly and assign a hallucination score between 0 and 1, where:

- 0.0: The OUTPUT is entirely faithful to the CONTEXT
- 1.0: The OUTPUT is entirely unfaithful to the CONTEXT

{examples_str}

INPUT (for context only, not to be used for faithfulness evaluation):
{input}

CONTEXT:
{context}

OUTPUT:
{output}

It is crucial that you provide your answer in the following JSON format:
{{
    "score": <your score between 0.0 and 1.0>,
    "reason": ["reason 1", "reason 2"]
}}
Reasons amount is not restricted. Output must be JSON format only.
```

🔗 Untuk informasi lebih lengkap tentang masing-masing metrik

* [Hallucination](https://www.comet.com/docs/opik/evaluation/metrics/hallucination)
* [Answer Relevance](https://www.comet.com/docs/opik/evaluation/metrics/answer_relevance)
* [Usefulness](https://www.comet.com/docs/opik/evaluation/metrics/usefulness)
* [Moderation](https://www.comet.com/docs/opik/evaluation/metrics/moderation)

---

### LLM as a Judges untuk model  (Base & Fine Tune)

In [ ]:
%pip install opik openai comet_ml litellm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 885.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.2/565.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import opik
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import (Hallucination, AnswerRelevance, Moderation, Usefulness)
from opik.integrations.openai import track_openai
import openai
import os
from datetime import datetime
from getpass import getpass
import litellm

# Define project name to enable tracing
os.environ["OPIK_PROJECT_NAME"] = "Eval LLM NLP"

In [ ]:
from google.colab import userdata

In [ ]:
# opik configs | set di secret colab
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = userdata.get('OPIK_API_KEY')

In [ ]:
workspace = 'final-projek-nlp'

In [ ]:
opik.configure(workspace= workspace)

OPIK: Opik is already configured. You can check the settings by viewing the config file at /root/.opik.config


In [ ]:
client = Opik()

In [ ]:
os.environ['OPENROUTER_API_KEY'] = userdata.get('OPENROUTER_API_KEY')

#### LLM Client

In [ ]:
class LLMClient:
    def __init__(self, client_type: str = "openai", model: str = "gpt-4"):
        self.client_type = client_type
        self.model = model

        if self.client_type == "openai":
            self.client = track_openai(openai.OpenAI())
        else:  # LiteLLM via OpenRouter
            self.client = track_openai(
                openai.OpenAI(
                    base_url="https://openrouter.ai/api/v1",
                    api_key=os.environ["OPENROUTER_API_KEY"]
                )
            )

    def _get_litellm_response(self, query: str, system: str = "You are a helpful assistant."):
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": query}
        ]
        response = litellm.completion(model=self.model, messages=messages)
        return response.choices[0].message["content"]

    def _get_openai_response(self, query: str, system: str = "You are a helpful assistant.", **kwargs):
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": query}
        ]
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            **kwargs
        )
        return response.choices[0].message.content

    def query(self, query: str, system: str = "You are a helpful assistant.", **kwargs):
        if self.client_type == 'openai':
            return self._get_openai_response(query, system, **kwargs)
        else:
            return self._get_litellm_response(query, system)

In [ ]:
# To set clinet
MODEL= 'openrouter/deepseek/deepseek-r1' # model bebas milih selama ada di OpenRouter
llm_client = LLMClient(model=MODEL, client_type='litellm')

In [ ]:
llm_client.query('Pagi')

'Selamat pagi! Ada yang bisa saya bantu hari ini?'

list-dataset

[link dataset opik](https://www.comet.com/opik/final-projek-nlp/datasets)

#### Dataset
Ini adalah dataset beris hasil ouput dari model base dan fine tune. Masing Masing dataset ini akan diberikan kepada LLM untuk penilaini

In [ ]:
list_dataset = ['Model-1 Base | unsloth/mistral-7b-v0.3',
 'Model-1 ft | unsloth/mistral-7b-v0.3',
 'Model-2 Base | unsloth/mistral-7b-v0.3',
 'Model-2 ft | unsloth/mistral-7b-v0.3',
 'Model-3 Base | unsloth/mistral-7b-instruct-v0.2-bnb-4bit',
 'Model-3 ft | unsloth/mistral-7b-instruct-v0.2-bnb-4bit',
 'Model-4 Base | unsloth/llama-3-8b-bnb-4bit',
 'Model-4 ft | unsloth/llama-3-8b-bnb-4bit',
 'Model-5 Base | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct',
 'Model-5 ft | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct',
 'Model-6 Base | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct',
 'Model-6 ft | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct',
 'Model-7 Base | Yellow-AI-NLP/komodo-7b-base',
 'Model-7 ft | Yellow-AI-NLP/komodo-7b-base',
 'Model-8 Base | Yellow-AI-NLP/komodo-7b-base',
 'Model-8 ft | Yellow-AI-NLP/komodo-7b-base']

In [ ]:
# Create or get the dataset
# contoh disini karena untuk model 1
dataset = client.get_or_create_dataset(name="Model-1 Base | unsloth/mistral-7b-v0.3")

#### Membuat  custom model

In [ ]:
from opik.evaluation.models import OpikBaseModel
from typing import List, Dict, Any

class CustomModel(OpikBaseModel):
    def __init__(self, model_name: str = "gpt-4", client_type: str = "openai"):
        super().__init__(model_name)
        self.client = LLMClient(client_type=client_type, model=model_name)

    def generate_provider_response(self, messages: List[Dict[str, Any]], **kwargs: Any) -> str:
        if not messages or messages[-1]["role"] != "user":
            raise ValueError("Last message must be from user.")
        system_message = messages[0]["content"] if messages[0]["role"] == "system" else "You are a helpful assistant."
        user_message = messages[-1]["content"]
        return self.client.query(user_message, system=system_message, **kwargs)

    def generate_string(self, input: str, **kwargs: Any) -> str:
        return self.client.query(input, **kwargs)

In [ ]:
def simple_evaluation_task(item):
    return {
        "input": item["input"],
        "output": item["prediction"],
        "context": item["reference"]  # reference jadi context untuk hallucination check misal
    }

In [ ]:
# Model Custom untuk evaluasi
MODEL = 'openrouter/deepseek/deepseek-r1:free'
custom_model = CustomModel(model_name=MODEL, client_type="litellm")

In [ ]:
from opik.evaluation.metrics import (Hallucination, AnswerRelevance, Moderation, Usefulness)

In [ ]:
# Hallucination metric
hallucination_metric = Hallucination(model=custom_model)
AnswerRelevanceMetric = AnswerRelevance(model=custom_model)
ModerationMetric = Moderation(model=custom_model)
UsefulnessMetric = Usefulness(model=custom_model)

In [ ]:
# Nama eksperimen
experiment_name = f"{list_dataset[0]} | {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} | model as judges evaluation: {MODEL}"
experiment_name

'Model-1 Base | unsloth/mistral-7b-v0.3 | 2025-05-31_10-09-42 | model as judges evaluation: openrouter/deepseek/deepseek-r1:free'

In [ ]:
# Run evaluation
evaluation = evaluate(
    experiment_name=experiment_name,
    dataset=dataset,
    task=simple_evaluation_task,
    scoring_metrics=[hallucination_metric, AnswerRelevanceMetric, ModerationMetric, UsefulnessMetric],
    experiment_config={"model": MODEL,
                       "dataset" : dataset.name},
    task_threads=10,
)

Evaluation:   0%|          | 0/139 [00:00<?, ?it/s]

OPIK: Failed to parse model output: Failed to extract presumably JSON dictionary: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/opik/evaluation/metrics/llm_judges/parsing_helpers.py", line 22, in _extract_presumably_json_dict_or_raise
    return json.loads(json_string)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most rec


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exc


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



...

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exce


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exc


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



KeyboardInterrupt: 

In [ ]:
# Nama eksperimen

MODEL = 'openrouter/openai/gpt-4o-mini'
custom_model = CustomModel(model_name=MODEL, client_type="litellm")

experiment_name = f"{list_dataset[0]} | {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} | model as judges evaluation: {MODEL}"
experiment_name

'Model-1 Base | unsloth/mistral-7b-v0.3 | 2025-05-31_03-42-25 | model as judges evaluation: openrouter/openai/gpt-4o-mini'

#### Function for evaluation

In [ ]:
from datetime import datetime

def run_evaluation(model_name, dataset,no_dataset, thread, sample= None):
    custom_model = CustomModel(model_name=model_name, client_type="litellm")

    # Metrik evaluasi
    hallucination = Hallucination(model=custom_model)
    relevance = AnswerRelevance(model=custom_model)
    moderation = Moderation(model=custom_model)
    usefulness = Usefulness(model=custom_model)

    # Nama eksperimen
    experiment_name = f"{list_dataset[no_dataset]} | {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} | model as judges evaluation: {model_name}"

    # Jalankan evaluasi
    evaluation = evaluate(
        experiment_name=experiment_name,
        dataset=dataset,
        task=simple_evaluation_task,
        scoring_metrics=[hallucination, relevance, moderation, usefulness],
        experiment_config={"model": model_name, "dataset": dataset.name},
        task_threads=thread,
        nb_samples=sample
    )

    return evaluation

### Evaluasi dengan LLM as judges

Berikut adalah satu contoh penerapannya

####  Base

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 0 # index
task_thread = 4
sample = None
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

Evaluation:   0%|          | 0/139 [00:00<?, ?it/s]

╭─ Model-1 Base | unsloth/mistral-7b-v0.3 (139 samples) ─╮
│                                                        │
│ Total time:        00:04:49                            │
│ Number of samples: 139                                 │
│                                                        │
│ hallucination_metric: 0.7525 (avg)                     │
│ answer_relevance_metric: 0.4665 (avg)                  │
│ moderation_metric: 0.0230 (avg)                        │
│ UsefulnessMetric: 0.2561 (avg)                         │
│                                                        │
╰────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=422756;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=01972484-ad58-7f93-87b0-5c405db20ba5&dataset_id=019723cb-8132-7b6c-8a09-5e865fd216c6&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

####  FT

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 1 # index
task_thread = 4
sample = None
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

Evaluation:   0%|          | 0/148 [00:00<?, ?it/s]

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-1 ft | unsloth/mistral-7b-v0.3 (148 samples) ─╮
│                                                      │
│ Total time:        00:05:58                          │
│ Number of samples: 148                               │
│                                                      │
│ hallucination_metric: 0.5020 (avg)                   │
│ answer_relevance_metric: 0.8291 (avg)                │
│ moderation_metric: 0.0020 (avg)                      │
│ UsefulnessMetric: 0.6871 (avg) - 1 failed            │
│                                                      │
╰──────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=188937;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019724a7-da3c-7662-9d3f-882ddfb4b6ec&dataset_id=019723cb-82d2-751c-86cb-ef9e0213ec6b&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Hasil dan kesimpulan

## Tabel Metrik Kuantitatif

|Model                                                        |bert_precision|bert_recall |bert_f1     |bleu1      |bleu2      |bleu3      |bleu4       |rouge1     |rouge2     |rougeL     |rougeLSum  |
|-------------------------------------------------------------|--------------|------------|------------|-----------|-----------|-----------|------------|-----------|-----------|-----------|-----------|
|Base Model 1 &#124; mistral-7b-v0.3                               |0.6211690002  |0.5893319779|0.6037431226|8.294900332|2.776794579|1.418423648|0.9493882534|11.42368319|1.42253989 |7.027056194|6.924935837|
|Finetuning Model Base 1 &#124; mistral-7b-v0.3                    |68.89656134   |68.59254551 |68.69144057 |26.40711447|12.6368507 |6.892742053|4.184926041 |30.29405405|6.696875565|15.46129952|15.46129952|
|Base Model 3 &#124; mistral-7b-instruct-v0.2-bnb-4bit             |0.6516675359  |0.6312426523|0.640989964 |12.96068764|4.302880714|2.145769235|1.413014908 |13.84793309|1.864717171|7.20778839 |7.849656376|
|Finetuning Model Base 3 &#124; mistral-7b-instruct-v0.2-bnb-4bit  |69.34224242   |69.16117907 |69.21357286 |27.09274326|13.04775774|7.174045826|4.48545549  |31.16239553|6.961255206|16.03152406|16.03152406|
|Base Model 4 &#124; llama-3-8b-bnb-4bit                           |0.6286781257  |0.6001730639|0.6130409542|8.522933346|2.880937301|1.473571158|0.9808751212|12.31186716|1.2991525  |7.692549568|7.344693805|
|Finetuning Model Base 4 &#124; llama-3-8b-bnb-4bit                |69.59667808   |69.19493806 |69.34840757 |26.15214464|12.57729233|6.848690689|4.149928476 |30.23881864|6.806361297|15.66878166|15.66878166|
|Base Model 5 &#124; llama3-8b-cpt-sahabatai-v1-instruct           |0.6811708496  |0.6525815765|0.6661404854|18.57281043|7.586095203|3.901354007|2.448177036 |25.70774977|4.680530871|13.40721771|16.23012061|
|Finetuning Model Base 5 &#124; llama3-8b-cpt-sahabatai-v1-instruct|69.75471719   |67.46975004 |68.56779786 |23.16172868|11.11923387|6.203401582|3.837907526 |27.61096927|6.217842509|14.46797133|14.68457661|
|Base Model 6 &#124; gemma2-9b-cpt-sahabatai-v1-instruct           |0.6815175981  |0.6697744528|0.6752528519|22.63963186|9.460875114|4.627718059|2.704908716 |29.47293451|5.410137253|14.84449991|18.23299866|
|Finetuning Model Base 6 &#124; gemma2-9b-cpt-sahabatai-v1-instruct|69.13028073   |69.10689962 |69.08257461 |27.26944609|13.11725512|7.260492495|4.473363274 |32.10991846|7.441202873|15.99050032|15.99050032|
|Base Model 7 &#124; Yellow-AI-NLP/komodo-7b-base                  |0.6343380072  |0.6379869936|0.6354655033|15.79090906|6.193109824|3.071179484|1.874798565 |20.21779641|3.355581782|10.55275484|13.15026689|
|Finetuning Model Base 7 &#124; Yellow-AI-NLP/komodo-7b-base       |68.5384351    |68.22293395 |68.32389957 |24.44578573|11.25720268|6.068727944|3.769305614 |29.04151225|6.365640991|15.21155278|15.21155278|
|Base Model 8 &#124; Yellow-AI-NLP/komodo-7b-base                  |0.6343380072  |0.6379869936|0.6354655033|15.79090906|6.193109824|3.071179484|1.874798565 |20.21779641|3.355581782|10.55275484|13.15026689|
|Finetuning Model Base 8 &#124; Yellow-AI-NLP/komodo-7b-base       |69.75249445   |69.73712081 |69.6947636  |28.3674836 |13.87887454|7.906611648|5.103033641 |32.21540203|7.605137453|16.79883638|16.79883638|
|Max                                                          |69.75471719   |69.73712081 |69.6947636  |28.3674836 |13.87887454|7.906611648|5.103033641 |32.21540203|7.605137453|16.79883638|18.23299866|


## Tabel Hasil Metrik LLM as a Judges

|No                                                           |Nama Model  |Tipe        |Hallucination (avg)|Answer Relevance (avg)|Moderation (avg)|Usefulness (avg)|Catatan Kegagalan Metrik|
|-------------------------------------------------------------|------------|------------|-------------------|----------------------|----------------|----------------|------------------------|
|1                                                            |unsloth/mistral-7b-v0.3|Base        |7,525              |4,665                 |0,0230          |2,561           |Halusinas tinggi, Answer Relevance rendah, usefulness rendah|
|1                                                            |unsloth/mistral-7b-v0.3|ft          |5,020              |8,291                 |0.020           |6,871           |- Halusianasi menurun- Answer Relevance  dan usefulness naik|
|3                                                            |unsloth/mistral-7b-instruct-v0.2-bnb-4bit|Base        |7,069              |7,718                 |0.0010          |6,257           |- Halusianasi menurun- Answer Relevance  dan usefulness naik|
|3                                                            |unsloth/mistral-7b-instruct-v0.2-bnb-4bit|ft          |4,870              |8,265                 |0,0080          |7,051           |- Halusianasi menurun- Answer Relevance  dan usefulness naik|
|4                                                            |unsloth/llama-3-8b-bnb-4bit|Base        |7,510              |5,640                 |0,0180          |3,400           |Halusinas tinggi, Answer Relevance rendah, usefulness rendah|
|4                                                            |unsloth/llama-3-8b-bnb-4bit|ft          |4,820              |8,320                 |0,0050          |6,990           |- Halusianasi menurun- Answer Relevance  dan usefulness naik|
|5                                                            |GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct|Base        |4,171              |8,543                 |0,0019          |7,327           |Moderation: 1 gagal, Usefulness: 1 gagal|
|5                                                            |GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct|ft          |4,950              |8,147                 |0               |6,408           |Hallucination: 2 gagal, Relevance: 1 gagal, Moderation: 1 gagal|
|6                                                            |GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct|Base        |3,284              |8,860                 |0               |7,931           |Relevance: 2 gagal, Usefulness: 1 gagal|
|6                                                            |GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct|ft          |5,120              |8,505                 |0,0120          |7,260           |-                       |
|7                                                            |Yellow-AI-NLP/komodo-7b-base|Base        |6,365              |7,218                 |0,0136          |4,904           |Relevance: 1 gagal, Moderation: 1 gagal|
|7                                                            |Yellow-AI-NLP/komodo-7b-base|ft          |5,770              |8,215                 |0,0070          |6,800           |- Halusianasi menurun- Answer Relevance  dan usefulness naik|
|8                                                            |Yellow-AI-NLP/komodo-7b-base|Base        |6,433              |7,250                 |0,0117          |4,788           |Relevance: 1 gagal, Moderation: 1 gagal|
|8                                                            |Yellow-AI-NLP/komodo-7b-base|ft          |4,667              |8,318                 |0,0020          |7,080           |- Halusianasi menurun- Answer Relevance  dan usefulness naik|


##Kesimpulan

* Penelitian ini  mengembangkan **chatbot kesehatan berbahasa Indonesia** yang berfokus pada sistem pencernaan, dengan pendekatan fine-tuning QLoRA terhadap 7 model LLM berbeda.
* Dataset yang digunakan berasal dari **Medical QA Indonesia** yang telah difilter khusus untuk domain pencernaan.
* Fine-tuning dengan QLoRA terbukti meningkatkan performa model, terutama dalam aspek **Answer Relevance**, **Usefulness**, dan **BERTScore F1**.
* Evaluasi kuantitatif (BLEU, ROUGE) menunjukkan peningkatan kemiripan jawaban terhadap referensi.
* Tidak semua model mengalami perbaikan; beberapa justru menunjukkan **peningkatan halusinasi** setelah fine-tuning (contoh: Model 4 dan 5).
* Model **komodo-7b-base** yang telah di-fine-tune menunjukkan **kinerja paling seimbang**, dengan tingkat relevansi dan usefulness tinggi serta halusinasi rendah, menjadikannya kandidat terbaik dalam penelitian ini.

---
